# MLCC Project Code for Ctrl+Alt+Deheat

Please take note that our tree based models were ran on Kaggle and our LSTM based ANNs were done locally/via Google Colab and Google Drive. If to recompile, multiple environemnt setup is required. Thus this merely serves as a placeholder for all our different notebooks. None of the code is actually compiled in this notebook here. For the figure and plotting results, please refer to the individual notebooks.

## Random Forest (Maize)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
import kagglehub
kagglehub.login()

In [ ]:
the_future_crop_challenge_path = kagglehub.competition_download('the-future-crop-challenge')
print("data import successfully")

In [ ]:
def load_data(crop: str, mode: str="train"):
    base_path = the_future_crop_challenge_path

    tasmax = pd.read_parquet(f"{base_path}/tasmax_{crop}_{mode}.parquet")
    tasmin = pd.read_parquet(f"{base_path}/tasmin_{crop}_{mode}.parquet")
    pr = pd.read_parquet(f"{base_path}/pr_{crop}_{mode}.parquet")
    rsds = pd.read_parquet(f"{base_path}/rsds_{crop}_{mode}.parquet")
    soil_co2 = pd.read_parquet(f"{base_path}/soil_co2_{crop}_{mode}.parquet")

    target = None
    if mode == "train":
        target = pd.read_parquet(f"{base_path}/train_solutions_{crop}.parquet")

    return {
        'tasmax': tasmax,
        'tasmin': tasmin,
        'pr': pr,
        'rsds': rsds,
        'soil_co2': soil_co2,
        'target': target,
    }

In [ ]:
# Load training data for maize
maize_train = load_data("maize", "train")

In [ ]:
def aggregate_daily_data(df):
    numeric_df = df.iloc[:, 4:].apply(pd.to_numeric, errors='coerce')
    return numeric_df.agg(['mean', 'max', 'min', 'std'], axis=1)
tasmax_agg_maize = aggregate_daily_data(maize_train['tasmax'])
tasmin_agg_maize = aggregate_daily_data(maize_train['tasmin'])
pr_agg_maize = aggregate_daily_data(maize_train['pr'])
rsds_agg_maize = aggregate_daily_data(maize_train['rsds'])
tasmax_agg_maize.columns = [f'tasmax_{col}' for col in tasmax_agg_maize.columns]
tasmin_agg_maize.columns = [f'tasmin_{col}' for col in tasmin_agg_maize.columns]
pr_agg_maize.columns = [f'pr_{col}' for col in pr_agg_maize.columns]
rsds_agg_maize.columns = [f'rsds_{col}' for col in rsds_agg_maize.columns]

feature_maize = pd.concat([tasmax_agg_maize, tasmin_agg_maize, pr_agg_maize, rsds_agg_maize, maize_train['soil_co2']], axis = 1)
target_maize = maize_train['target']

In [ ]:
feature_maize

In [ ]:
# Drop non-numeric columns
numeric_features_maize = feature_maize.select_dtypes(include=[np.number])

# Scale features
scaler_maize = StandardScaler()
features_scaled_maize = scaler_maize.fit_transform(numeric_features_maize)

In [ ]:
year_series = maize_train['tasmax']['year']

In [ ]:
# Step 2: Find validation year (2020)
val_year = year_series.max()  # Should be 2020
print(f"Using year {val_year} for validation.") # Note that this printout does not resemble the actual year of 2020

In [ ]:
val_mask = (year_series == val_year)

# Step 4: Split data
X_train = features_scaled_maize[~val_mask]
X_val = features_scaled_maize[val_mask]
y_train = target_maize[~val_mask].values.ravel()
y_val = target_maize[val_mask].values.ravel()

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

In [ ]:
y_val_pred = rf_model.predict(X_val)

# Step 7: Evaluation
from sklearn.metrics import mean_squared_error, r2_score

rmse = mean_squared_error(y_val, y_val_pred, squared=False)
r2 = r2_score(y_val, y_val_pred)

print(f"Random Forest Validation RMSE (2020): {rmse:.4f}")
print(f"Random Forest Validation R2 (2020): {r2:.4f}")

In [ ]:
# Predicted vs Actual Scatter Plot (for 2020)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_val, y=y_val_pred, alpha=0.6, edgecolor=None)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', label='Perfect Prediction')
plt.xlabel("Actual Yield (2020)")
plt.ylabel("Predicted Yield (2020)")
plt.title(f" Random Forest: Predicted vs Actual Maize Yield (2020)\nR² = {r2:.3f}, RMSE = {rmse:.3f}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Residuals (Actual - Predicted)
residuals = y_val - y_val_pred
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color='orange')
plt.title("Random Forest Residuals (2020): Actual - Predicted (Maize)")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Load test data
maize_test = load_data("maize", "test")

In [ ]:
maize_test = load_data("maize", "test")

tasmax_agg_test = aggregate_daily_data(maize_test['tasmax'])
tasmax_agg_test.columns = [f'tasmax_{col}' for col in tasmax_agg_test.columns]

tasmin_agg_test = aggregate_daily_data(maize_test['tasmin'])
tasmin_agg_test.columns = [f'tasmin_{col}' for col in tasmin_agg_test.columns]

pr_agg_test = aggregate_daily_data(maize_test['pr'])
pr_agg_test.columns = [f'pr_{col}' for col in pr_agg_test.columns]

rsds_agg_test = aggregate_daily_data(maize_test['rsds'])
rsds_agg_test.columns = [f'rsds_{col}' for col in rsds_agg_test.columns]

features_test = pd.concat([tasmax_agg_test, tasmin_agg_test, pr_agg_test, rsds_agg_test, maize_test['soil_co2']], axis=1)

numeric_features_test = features_test.select_dtypes(include=[np.number])

numeric_features_test = numeric_features_test.dropna()

features_scaled_test = scaler_maize.transform(numeric_features_test)
test_ids = numeric_features_test.index

In [ ]:
rf_predictions = rf_model.predict(features_scaled_test)
rf_results = pd.DataFrame({
    "ID": test_ids,
    "Predicted_Yield": rf_predictions
})
rf_results.head()

In [ ]:
# Step 0: Year decoder function
def decode_year(encoded_year):
    return encoded_year - 381 + 1980

# Step 1: Random Forest predictions (2021–2100), with decoded years
rf_years_decoded = decode_year(maize_test['tasmax'].loc[numeric_features_test.index, 'year']).astype(int)

rf_pred_df = pd.DataFrame({
    "Year": rf_years_decoded,
    "Yield": rf_predictions
}, index=numeric_features_test.index)

rf_future = rf_pred_df[(rf_pred_df["Year"] >= 2021) & (rf_pred_df["Year"] <= 2100)]
rf_future_avg = rf_future.groupby("Year", as_index=False)["Yield"].mean().rename(columns={"Yield": "Yield_pred"})

# Step 2: Historical yield (1980–2020), reuse val_mask and y_train/y_val from earlier RF setup
encoded_years = maize_train["tasmax"]["year"]

# Decode training and validation years
train_years = decode_year(encoded_years[~val_mask].astype(int))
val_years = decode_year(encoded_years[val_mask].astype(int))

historical_years_all = pd.concat([train_years.reset_index(drop=True), val_years.reset_index(drop=True)])
historical_yields_all = np.concatenate([y_train, y_val])

historical_df = pd.DataFrame({
    "Year": historical_years_all,
    "Yield_hist": historical_yields_all
})
historical_avg = historical_df.groupby("Year", as_index=False)["Yield_hist"].mean()

# Step 3: Merge into full 1980–2100 DataFrame
all_years = pd.DataFrame({"Year": range(1980, 2101)})
historical_avg["Year"] = historical_avg["Year"].astype(int)
rf_future_avg["Year"] = rf_future_avg["Year"].astype(int)

plot_df = all_years.merge(historical_avg, on="Year", how="left")
plot_df = plot_df.merge(rf_future_avg, on="Year", how="left")

# Step 4: Plot Random Forest line (same style as XGBoost version)
plt.figure(figsize=(16, 6))
plt.plot(plot_df["Year"][plot_df["Yield_hist"].notna()],
         plot_df["Yield_hist"].dropna(),
         marker="o", color="tab:blue", label="Historical Yield (1980–2020)")

plt.plot(plot_df["Year"][plot_df["Yield_pred"].notna()],
         plot_df["Yield_pred"].dropna(),
         marker="o", color="tab:orange", label="Random Forest Predicted Yield (2021–2100)")

plt.xlabel("Year")
plt.ylabel("Mean yield across highly-harvested gridcells (>2000 hectares)")
plt.title("Random Forest Maize Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.xticks(range(1980, 2101, 10))
plt.ylim(2.96)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Combine historical and predicted yields into one continuous series
combined_yield = pd.concat([
    plot_df.set_index("Year")["Yield_hist"].dropna(),
    plot_df.set_index("Year")["Yield_pred"].dropna()
])

# Plot the full timeline
plt.figure(figsize=(16, 6))
plt.plot(combined_yield.index, combined_yield.values, marker="o", color="tab:purple", label="Random Forest: Historical + Predicted Yield")
plt.xlabel("Year")
plt.ylabel("Mean yield (t/ha) across highly-harvested gridcells (>2000 hectares)")
plt.title("Random Forest Maize Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.ylim(3.0, combined_yield.max() * 1.05)  # Start y-axis at 3.0
plt.xticks(range(1980, 2101, 10))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance from XGBoost
importances = rf_model.feature_importances_
features = numeric_features_maize.columns
sorted_idx = np.argsort(importances)[::-1]  # Descending order
top_idx = sorted_idx[:10]

plt.figure(figsize=(10, 5))
sns.barplot(x=importances[top_idx], y=features[top_idx], ci=None)
plt.title("Feature Importance from Random Forest(Maize)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
importances = rf_model.feature_importances_
features = numeric_features_maize.columns

# Create a DataFrame of features and their importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
})

# Sort by importance descending
importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Display top N (optional)
importance_df.head(20)

In [ ]:
# Prepare historical and predicted yield DataFrames (same structure as XGBoost version)
historical_rf_df = historical_df[["Year", "Yield_hist"]].rename(columns={"Yield_hist": "Yield"})
historical_rf_df = historical_rf_df[(historical_rf_df["Year"] >= 1980) & (historical_rf_df["Year"] <= 2020)]

future_rf_df = rf_pred_df[rf_pred_df["Year"] >= 2021]

# KDE plot for yield distribution
plt.figure(figsize=(8, 5))
sns.kdeplot(data=historical_rf_df, x="Yield", label="1980–2020", fill=True)
sns.kdeplot(data=future_rf_df, x="Yield", label="2021–2100", fill=True)
plt.title("Maize Yield Distribution: Historical vs Predicted (Random Forest)")
plt.xlabel("Yield")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

## Random Forest (Wheat)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

def load_data(crop: str, mode: str="train"):
    base_path = "/kaggle/input/the-future-crop-challenge"

    tasmax = pd.read_parquet(f"{base_path}/tasmax_{crop}_{mode}.parquet")
    tasmin = pd.read_parquet(f"{base_path}/tasmin_{crop}_{mode}.parquet")
    pr = pd.read_parquet(f"{base_path}/pr_{crop}_{mode}.parquet")
    rsds = pd.read_parquet(f"{base_path}/rsds_{crop}_{mode}.parquet")
    soil_co2 = pd.read_parquet(f"{base_path}/soil_co2_{crop}_{mode}.parquet")

    target = None
    if mode == "train":
        target = pd.read_parquet(f"{base_path}/train_solutions_{crop}.parquet")

    return {
        'tasmax': tasmax,
        'tasmin': tasmin,
        'pr': pr,
        'rsds': rsds,
        'soil_co2': soil_co2,
        'target': target,
    }

wheat_train = load_data("wheat", "train")

In [ ]:
wheat_train['tasmax'].head()

In [ ]:
def aggregate_daily_data(df):
    numeric_df = df.iloc[:, 4:].apply(pd.to_numeric, errors='coerce')
    return numeric_df.agg(['mean', 'max', 'min', 'std'], axis=1)

tasmax_agg_wheat = aggregate_daily_data(wheat_train['tasmax'])
tasmin_agg_wheat = aggregate_daily_data(wheat_train['tasmin'])
pr_agg_wheat = aggregate_daily_data(wheat_train['pr'])
rsds_agg_wheat = aggregate_daily_data(wheat_train['rsds'])

tasmax_agg_wheat.columns = [f'tasmax_{col}' for col in tasmax_agg_wheat.columns]
tasmin_agg_wheat.columns = [f'tasmin_{col}' for col in tasmin_agg_wheat.columns]
pr_agg_wheat.columns     = [f'pr_{col}' for col in pr_agg_wheat.columns]
rsds_agg_wheat.columns   = [f'rsds_{col}' for col in rsds_agg_wheat.columns]

features_wheat = pd.concat([tasmax_agg_wheat, tasmin_agg_wheat, pr_agg_wheat, rsds_agg_wheat, wheat_train['soil_co2']], axis=1)
target_wheat = wheat_train['target']

In [ ]:
features_wheat

In [ ]:
target_wheat

In [ ]:
numeric_features_wheat = features_wheat.select_dtypes(include=[np.number])

scaler_wheat = StandardScaler()
features_scaled_wheat = scaler_wheat.fit_transform(numeric_features_wheat)

In [ ]:
year_series = wheat_train['tasmax']['year']
val_year = year_series.max()  # Should be 2020
print(f"Using year {val_year} for validation.")

In [ ]:
val_mask = (year_series == val_year)
X_train = features_scaled_wheat[~val_mask]
X_val = features_scaled_wheat[val_mask]
y_train = target_wheat[~val_mask].values.ravel()
y_val = target_wheat[val_mask].values.ravel()

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# from tqdm import tqdm

# model = RandomForestRegressor(
#     n_estimators=1,
#     warm_start=True,
#     random_state=42
# )

# for i in tqdm(range(1, 101), desc="Training Random Forest"):
#     model.set_params(n_estimators=i)
#     model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
y_val_pred = model.predict(X_val)

rmse = mean_squared_error(y_val, y_val_pred, squared=False)
r2 = r2_score(y_val, y_val_pred)

print(f"Random Forest Validation RMSE (2020): {rmse:.4f}")
print(f"Random Forest Validation R2 (2020): {r2:.4f}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_val, y=y_val_pred, alpha=0.6, edgecolor=None)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', label='Perfect Prediction')
plt.xlabel("Actual Yield (2020)")
plt.ylabel("Predicted Yield (2020)")
plt.title(f" Random Forest: Predicted vs Actual Wheat Yield (2020)\nR² = {r2:.3f}, RMSE = {rmse:.3f}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Residuals (Actual - Predicted)
residuals = y_val - y_val_pred
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color='orange')
plt.title("Random Forest Residuals (2020): Actual - Predicted (Wheat)")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Load test data and do the same
wheat_test = load_data("wheat", "test")

tasmax_agg_test = aggregate_daily_data(wheat_test['tasmax'])
tasmax_agg_test.columns = [f'tasmax_{col}' for col in tasmax_agg_test.columns]

tasmin_agg_test = aggregate_daily_data(wheat_test['tasmin'])
tasmin_agg_test.columns = [f'tasmin_{col}' for col in tasmin_agg_test.columns]

pr_agg_test = aggregate_daily_data(wheat_test['pr'])
pr_agg_test.columns = [f'pr_{col}' for col in pr_agg_test.columns]

rsds_agg_test = aggregate_daily_data(wheat_test['rsds'])
rsds_agg_test.columns = [f'rsds_{col}' for col in rsds_agg_test.columns]

features_test = pd.concat([tasmax_agg_test, tasmin_agg_test, pr_agg_test, rsds_agg_test, wheat_test['soil_co2']], axis=1)

numeric_features_test = features_test.select_dtypes(include=[np.number])

numeric_features_test = numeric_features_test.dropna()

features_scaled_test = scaler_wheat.transform(numeric_features_test)
test_ids = numeric_features_test.index

predictions = model.predict(features_scaled_test)
pred_results = pd.DataFrame({
    "ID": test_ids,
    "Predicted_Yield": predictions
})
pred_results.head()

In [ ]:
# Step 0: Year decoder function
def decode_year(encoded_year):
    return encoded_year - 381 + 1980

# Step 1: Random Forest predictions (2021–2100), with decoded years
rf_years_decoded = decode_year(wheat_test['tasmax'].loc[numeric_features_test.index, 'year']).astype(int)

rf_pred_df = pd.DataFrame({
    "Year": rf_years_decoded,
    "Yield": predictions
}, index=numeric_features_test.index)

rf_future = rf_pred_df[(rf_pred_df["Year"] >= 2021) & (rf_pred_df["Year"] <= 2100)]
rf_future_avg = rf_future.groupby("Year", as_index=False)["Yield"].mean().rename(columns={"Yield": "Yield_pred"})

# Step 2: Historical yield (1980–2020), reuse val_mask and y_train/y_val from earlier RF setup
encoded_years = wheat_train["tasmax"]["year"]

# Decode training and validation years
train_years = decode_year(encoded_years[~val_mask].astype(int))
val_years = decode_year(encoded_years[val_mask].astype(int))

historical_years_all = pd.concat([train_years.reset_index(drop=True), val_years.reset_index(drop=True)])
historical_yields_all = np.concatenate([y_train, y_val])

historical_df = pd.DataFrame({
    "Year": historical_years_all,
    "Yield_hist": historical_yields_all
})
historical_avg = historical_df.groupby("Year", as_index=False)["Yield_hist"].mean()

# Step 3: Merge into full 1980–2100 DataFrame
all_years = pd.DataFrame({"Year": range(1980, 2101)})
historical_avg["Year"] = historical_avg["Year"].astype(int)
rf_future_avg["Year"] = rf_future_avg["Year"].astype(int)

plot_df = all_years.merge(historical_avg, on="Year", how="left")
plot_df = plot_df.merge(rf_future_avg, on="Year", how="left")

# Step 4: Plot Random Forest line (same style as XGBoost version)
plt.figure(figsize=(16, 6))
plt.plot(plot_df["Year"][plot_df["Yield_hist"].notna()],
         plot_df["Yield_hist"].dropna(),
         marker="o", color="tab:blue", label="Historical Yield (1980–2020)")

plt.plot(plot_df["Year"][plot_df["Yield_pred"].notna()],
         plot_df["Yield_pred"].dropna(),
         marker="o", color="tab:orange", label="Random Forest Predicted Yield (2021–2100)")

plt.xlabel("Year")
plt.ylabel("Mean yield across highly-harvested gridcells (>2000 hectares)")
plt.title("Random Forest Wheat Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.xticks(range(1980, 2101, 10))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Combine historical and predicted yields into one continuous series
combined_yield = pd.concat([
    plot_df.set_index("Year")["Yield_hist"].dropna(),
    plot_df.set_index("Year")["Yield_pred"].dropna()
])

# Plot the full timeline
plt.figure(figsize=(16, 6))
plt.plot(combined_yield.index, combined_yield.values, marker="o", color="tab:purple", label="Random Forest: Historical + Predicted Yield")
plt.xlabel("Year")
plt.ylabel("Mean yield (t/ha) across highly-harvested gridcells (>2000 hectares)")
plt.title("Random Forest Wheat Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.xticks(range(1980, 2101, 10))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance from XGBoost
importances = model.feature_importances_
features = numeric_features_wheat.columns
sorted_idx = np.argsort(importances)[::-1]  # Descending order
top_idx = sorted_idx[:10]

plt.figure(figsize=(10, 5))
sns.barplot(x=importances[top_idx], y=features[top_idx], ci=None)
plt.title("Feature Importance from Random Forest(Wheat)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
# Prepare historical and predicted yield DataFrames (same structure as XGBoost version)
historical_rf_df = historical_df[["Year", "Yield_hist"]].rename(columns={"Yield_hist": "Yield"})
historical_rf_df = historical_rf_df[(historical_rf_df["Year"] >= 1980) & (historical_rf_df["Year"] <= 2020)]

future_rf_df = rf_pred_df[rf_pred_df["Year"] >= 2021]

# KDE plot for yield distribution
plt.figure(figsize=(8, 5))
sns.kdeplot(data=historical_rf_df, x="Yield", label="1980–2020", fill=True)
sns.kdeplot(data=future_rf_df, x="Yield", label="2021–2100", fill=True)
plt.title("Yield Distribution: Historical vs Predicted (Random Forest)")
plt.xlabel("Yield (t/ha)")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

## XGBoost (Maize)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

In [ ]:
import kagglehub
kagglehub.login()

In [ ]:
the_future_crop_challenge_path = kagglehub.competition_download('the-future-crop-challenge')
print("data import successfully")

In [ ]:
def load_data(crop: str, mode: str="train"):
    base_path = the_future_crop_challenge_path

    tasmax = pd.read_parquet(f"{base_path}/tasmax_{crop}_{mode}.parquet")
    tasmin = pd.read_parquet(f"{base_path}/tasmin_{crop}_{mode}.parquet")
    pr = pd.read_parquet(f"{base_path}/pr_{crop}_{mode}.parquet")
    rsds = pd.read_parquet(f"{base_path}/rsds_{crop}_{mode}.parquet")
    soil_co2 = pd.read_parquet(f"{base_path}/soil_co2_{crop}_{mode}.parquet")

    target = None
    if mode == "train":
        target = pd.read_parquet(f"{base_path}/train_solutions_{crop}.parquet")

    return {
        'tasmax': tasmax,
        'tasmin': tasmin,
        'pr': pr,
        'rsds': rsds,
        'soil_co2': soil_co2,
        'target': target,
    }

In [ ]:
feature_maize

In [ ]:
# Load training data for maize
maize_train = load_data("maize", "train")

In [ ]:
def aggregate_daily_data(df):
    numeric_df = df.iloc[:, 4:].apply(pd.to_numeric, errors='coerce')
    return numeric_df.agg(['mean', 'max', 'min', 'std'], axis=1)
tasmax_agg_maize = aggregate_daily_data(maize_train['tasmax'])
tasmin_agg_maize = aggregate_daily_data(maize_train['tasmin'])
pr_agg_maize = aggregate_daily_data(maize_train['pr'])
rsds_agg_maize = aggregate_daily_data(maize_train['rsds'])
tasmax_agg_maize.columns = [f'tasmax_{col}' for col in tasmax_agg_maize.columns]
tasmin_agg_maize.columns = [f'tasmin_{col}' for col in tasmin_agg_maize.columns]
pr_agg_maize.columns = [f'pr_{col}' for col in pr_agg_maize.columns]
rsds_agg_maize.columns = [f'rsds_{col}' for col in rsds_agg_maize.columns]

feature_maize = pd.concat([tasmax_agg_maize, tasmin_agg_maize, pr_agg_maize, rsds_agg_maize, maize_train['soil_co2']], axis = 1)
target_maize = maize_train['target']

In [ ]:
# Drop non-numeric columns
numeric_features_maize = feature_maize.select_dtypes(include=[np.number])

# Scale features
scaler_maize = StandardScaler()
features_scaled_maize = scaler_maize.fit_transform(numeric_features_maize)

In [ ]:
# Get the year column
year_series = maize_train['tasmax']['year']

# Use the maximum year as validation year
val_year = year_series.max()
print(f"Using year {val_year} as validation set") # Note that again this is not printing out the real year of 2020

# Create boolean mask for validation
val_mask = (year_series == val_year)

# Split data
X_train = features_scaled_maize[~val_mask]
X_val = features_scaled_maize[val_mask]
y_train = target_maize[~val_mask].values.ravel()
y_val = target_maize[val_mask].values.ravel()

In [ ]:
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
)

In [ ]:
xgb_model.fit(X_train, y_train)
# Predict
y_pred = xgb_model.predict(X_val)
# Evaluation
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)
print(f"XGBoost Validation RMSE: {rmse:.4f}")
print(f"XGBoost Validation R2: {r2:.4f}")

In [ ]:
# Plot Predicted vs Actual
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_val, y=y_pred, alpha=0.6, edgecolor=None)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', label='Perfect Prediction')
plt.xlabel("Actual Yield")
plt.ylabel("Predicted Yield")
plt.title(f"XGBoost Predicted vs Actual Maize Yield (2020)\nR² = {r2:.3f}, RMSE = {rmse:.3f}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
residuals = y_val - y_pred

plt.figure(figsize=(8,6))
sns.histplot(residuals, bins=30, kde=True, color='green')
plt.title("XGBoost Residuals (2020): Actual - Predicted (Maize)")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Load test data
maize_test = load_data("maize", "test")

tasmax_agg_test = aggregate_daily_data(maize_test['tasmax'])
tasmax_agg_test.columns = [f'tasmax_{col}' for col in tasmax_agg_test.columns]

tasmin_agg_test = aggregate_daily_data(maize_test['tasmin'])
tasmin_agg_test.columns = [f'tasmin_{col}' for col in tasmin_agg_test.columns]

pr_agg_test = aggregate_daily_data(maize_test['pr'])
pr_agg_test.columns = [f'pr_{col}' for col in pr_agg_test.columns]

rsds_agg_test = aggregate_daily_data(maize_test['rsds'])
rsds_agg_test.columns = [f'rsds_{col}' for col in rsds_agg_test.columns]

features_test = pd.concat([tasmax_agg_test, tasmin_agg_test, pr_agg_test, rsds_agg_test, maize_test['soil_co2']], axis=1)

numeric_features_test = features_test.select_dtypes(include=[np.number])

numeric_features_test = numeric_features_test.dropna()

features_scaled_test = scaler_maize.transform(numeric_features_test)
test_ids = numeric_features_test.index

In [ ]:
xgb_predictions = xgb_model.predict(features_scaled_test)

# Store predictions with index
xgb_results = pd.DataFrame({
    'ID': numeric_features_test.index,
    'Predicted_Yield': xgb_predictions
})

xgb_results.head()

In [ ]:
# Step 0: Year decoder function
def decode_year(encoded_year):
    return encoded_year - 381 + 1980

# Step 1: XGBoost predictions (2021–2100), with decoded years
xgb_pred_df = pd.DataFrame({
    "Year": decode_year(maize_test['tasmax'].loc[numeric_features_test.index, 'year']).astype(int),
    "Yield": xgb_predictions
}, index=numeric_features_test.index)

xgb_future = xgb_pred_df[(xgb_pred_df["Year"] >= 2021) & (xgb_pred_df["Year"] <= 2100)]
xgb_future_avg = xgb_future.groupby("Year", as_index=False)["Yield"].mean().rename(columns={"Yield": "Yield_pred"})

# Step 2: Historical yield (1980–2020 including validation), with decoded years
encoded_years = maize_train["tasmax"]["year"]

# Use val_mask to split years for train and val
train_years = decode_year(encoded_years[~val_mask].astype(int))
val_years = decode_year(encoded_years[val_mask].astype(int))

historical_years_all = pd.concat([train_years.reset_index(drop=True), val_years.reset_index(drop=True)])
historical_yields_all = np.concatenate([y_train, y_val])

historical_df = pd.DataFrame({
    "Year": historical_years_all,
    "Yield_hist": historical_yields_all
})
historical_avg = historical_df.groupby("Year", as_index=False)["Yield_hist"].mean()

# Step 3: Combine into full plot DataFrame (1980–2100)
all_years = pd.DataFrame({"Year": range(1980, 2101)})
historical_avg["Year"] = historical_avg["Year"].astype(int)
xgb_future_avg["Year"] = xgb_future_avg["Year"].astype(int)

plot_df = all_years.merge(historical_avg, on="Year", how="left")
plot_df = plot_df.merge(xgb_future_avg, on="Year", how="left")

# Step 4: Plot
plt.figure(figsize=(16, 6))
plt.plot(plot_df["Year"][plot_df["Yield_hist"].notna()],
         plot_df["Yield_hist"].dropna(),
         marker="o", color="tab:blue", label="Historical Yield (1980–2020)")

plt.plot(plot_df["Year"][plot_df["Yield_pred"].notna()],
         plot_df["Yield_pred"].dropna(),
         marker="o", color="tab:green", label="XGBoost Predicted Yield (2021–2100)")
plt.xlabel("Year")
plt.ylabel("Mean yield across highly-harvested gridcells (>2000 hectares)")
plt.title("Maize Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.xticks(range(1980, 2101, 10))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Combine historical + predicted into one series
combined_yield = pd.concat([
    plot_df.set_index("Year")["Yield_hist"].dropna(),
    plot_df.set_index("Year")["Yield_pred"].dropna()
])

plt.figure(figsize=(16, 6))
plt.plot(combined_yield.index, combined_yield.values, marker="o", color="tab:purple", label="Historical + Predicted Yield")
plt.xlabel("Year")
plt.ylabel("Mean yield (t/ha)")
plt.title("Maize Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.xticks(range(1980, 2101, 10))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance from XGBoost
importances = xgb_model.feature_importances_
features = numeric_features_maize.columns
sorted_idx = np.argsort(importances)[::-1]  # Descending order
top_idx = sorted_idx[:10]

plt.figure(figsize=(10, 5))
sns.barplot(x=importances[top_idx], y=features[top_idx], ci=None)
plt.title("Feature Importance from XGBoost(Maize)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
importances = xgb_model.feature_importances_
features = numeric_features_maize.columns

# Create a DataFrame of features and their importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
})

# Sort by importance descending
importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Display top N (optional)
importance_df.head(20)

In [ ]:
# Prepare historical and predicted yield DataFrames
historical_xgb_df = historical_df[["Year", "Yield_hist"]].rename(columns={"Yield_hist": "Yield"})
historical_xgb_df = historical_xgb_df[(historical_xgb_df["Year"] >= 1980) & (historical_xgb_df["Year"] <= 2020)]

future_xgb_df = xgb_pred_df[xgb_pred_df["Year"] >= 2021]

# KDE plot for yield distribution
plt.figure(figsize=(8, 5))
sns.kdeplot(data=historical_xgb_df, x="Yield", label="1980–2020", fill=True)
sns.kdeplot(data=future_xgb_df, x="Yield", label="2021–2100", fill=True)
plt.title("Maize Yield Distribution: Historical vs Predicted (XGBoost)")
plt.xlabel("Yield")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

## XGBoost (Wheat)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

def load_data(crop: str, mode: str="train"):
    base_path = "/kaggle/input/the-future-crop-challenge"

    tasmax = pd.read_parquet(f"{base_path}/tasmax_{crop}_{mode}.parquet")
    tasmin = pd.read_parquet(f"{base_path}/tasmin_{crop}_{mode}.parquet")
    pr = pd.read_parquet(f"{base_path}/pr_{crop}_{mode}.parquet")
    rsds = pd.read_parquet(f"{base_path}/rsds_{crop}_{mode}.parquet")
    soil_co2 = pd.read_parquet(f"{base_path}/soil_co2_{crop}_{mode}.parquet")

    target = None
    if mode == "train":
        target = pd.read_parquet(f"{base_path}/train_solutions_{crop}.parquet")

    return {
        'tasmax': tasmax,
        'tasmin': tasmin,
        'pr': pr,
        'rsds': rsds,
        'soil_co2': soil_co2,
        'target': target,
    }

wheat_train = load_data("wheat", "train")

In [ ]:
wheat_train['soil_co2']

In [ ]:
wheat_train['tasmax']

In [ ]:
def aggregate_daily_data(df):
    numeric_df = df.iloc[:, 4:].apply(pd.to_numeric, errors='coerce')
    return numeric_df.agg(['mean', 'max', 'min', 'std'], axis=1)

tasmax_agg_wheat = aggregate_daily_data(wheat_train['tasmax'])
tasmin_agg_wheat = aggregate_daily_data(wheat_train['tasmin'])
pr_agg_wheat = aggregate_daily_data(wheat_train['pr'])
rsds_agg_wheat = aggregate_daily_data(wheat_train['rsds'])

tasmax_agg_wheat.columns = [f'tasmax_{col}' for col in tasmax_agg_wheat.columns]
tasmin_agg_wheat.columns = [f'tasmin_{col}' for col in tasmin_agg_wheat.columns]
pr_agg_wheat.columns     = [f'pr_{col}' for col in pr_agg_wheat.columns]
rsds_agg_wheat.columns   = [f'rsds_{col}' for col in rsds_agg_wheat.columns]

features_wheat = pd.concat([tasmax_agg_wheat, tasmin_agg_wheat, pr_agg_wheat, rsds_agg_wheat, wheat_train['soil_co2']], axis=1)
target_wheat = wheat_train['target']

In [ ]:
target_wheat

In [ ]:
features_wheat['texture_class']

In [ ]:
print(features_wheat.columns.tolist())

In [ ]:
numeric_features_wheat = features_wheat.select_dtypes(include=[np.number])
scaler_wheat = StandardScaler()
features_scaled_wheat = scaler_wheat.fit_transform(numeric_features_wheat)

In [ ]:
year_series = wheat_train['tasmax']['year']

val_year = year_series.max()
print(f"Using year {val_year} as validation set") #Note that here again, this is not the actual year of 2020

val_mask = (year_series == val_year)

# Split data
X_train = features_scaled_wheat[~val_mask]
X_val = features_scaled_wheat[val_mask]
y_train = target_wheat[~val_mask].values.ravel()
y_val = target_wheat[val_mask].values.ravel()

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error,r2_score

xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)
print(f"XGBoost Validation RMSE: {rmse:.4f}")
print(f"XGBoost Validation R2: {r2:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_val, y=y_pred)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', label='Perfect Prediction')
plt.xlabel("Actual Yield")
plt.ylabel("Predicted Yield")
plt.title(f"XGBoost Predicted vs Actual Wheat Yield (Validation Set)\nR² = {r2:.3f}, RMSE = {rmse:.3f}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
residuals = y_val - y_pred

plt.figure(figsize=(8,6))
sns.histplot(residuals, bins=30, kde=True, color='green')
plt.title("XGBoost Residual Distribution: Actual - Predicted(Wheat)")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
wheat_test = load_data("wheat", "test")

tasmax_agg_test = aggregate_daily_data(wheat_test['tasmax'])
tasmax_agg_test.columns = [f'tasmax_{col}' for col in tasmax_agg_test.columns]

tasmin_agg_test = aggregate_daily_data(wheat_test['tasmin'])
tasmin_agg_test.columns = [f'tasmin_{col}' for col in tasmin_agg_test.columns]

pr_agg_test = aggregate_daily_data(wheat_test['pr'])
pr_agg_test.columns = [f'pr_{col}' for col in pr_agg_test.columns]

rsds_agg_test = aggregate_daily_data(wheat_test['rsds'])
rsds_agg_test.columns = [f'rsds_{col}' for col in rsds_agg_test.columns]


features_test = pd.concat([tasmax_agg_test, tasmin_agg_test, pr_agg_test, rsds_agg_test, wheat_test['soil_co2']], axis=1)

numeric_features_test = features_test.select_dtypes(include=[np.number]).dropna()

features_scaled_test = scaler_wheat.transform(numeric_features_test)

xgb_predictions = xgb_model.predict(features_scaled_test)

xgb_results = pd.DataFrame({
    'ID': numeric_features_test.index,
    'Predicted_Yield': xgb_predictions
})

xgb_results.head()

In [ ]:
# Step 0: Year decoder function
def decode_year(encoded_year):
    return encoded_year - 381 + 1980

# Step 1: XGBoost predictions (2021–2100), with decoded years
xgb_pred_df = pd.DataFrame({
    "Year": decode_year(wheat_test['tasmax'].loc[numeric_features_test.index, 'year']).astype(int),
    "Yield": xgb_predictions
}, index=numeric_features_test.index)

xgb_future = xgb_pred_df[(xgb_pred_df["Year"] >= 2021) & (xgb_pred_df["Year"] <= 2100)]
xgb_future_avg = xgb_future.groupby("Year", as_index=False)["Yield"].mean().rename(columns={"Yield": "Yield_pred"})

# Step 2: Historical yield (1980–2020 including validation), with decoded years
encoded_years = wheat_train["tasmax"]["year"]

# Use val_mask to split years for train and val
train_years = decode_year(encoded_years[~val_mask].astype(int))
val_years = decode_year(encoded_years[val_mask].astype(int))

historical_years_all = pd.concat([train_years.reset_index(drop=True), val_years.reset_index(drop=True)])
historical_yields_all = np.concatenate([y_train, y_val])

historical_df = pd.DataFrame({
    "Year": historical_years_all,
    "Yield_hist": historical_yields_all
})
historical_avg = historical_df.groupby("Year", as_index=False)["Yield_hist"].mean()

# Step 3: Combine into full plot DataFrame (1980–2100)
all_years = pd.DataFrame({"Year": range(1980, 2101)})
historical_avg["Year"] = historical_avg["Year"].astype(int)
xgb_future_avg["Year"] = xgb_future_avg["Year"].astype(int)

plot_df = all_years.merge(historical_avg, on="Year", how="left")
plot_df = plot_df.merge(xgb_future_avg, on="Year", how="left")

# Step 4: Plot
plt.figure(figsize=(16, 6))
plt.plot(plot_df["Year"][plot_df["Yield_hist"].notna()],
         plot_df["Yield_hist"].dropna(),
         marker="o", color="tab:blue", label="Historical Yield (1980–2020)")

plt.plot(plot_df["Year"][plot_df["Yield_pred"].notna()],
         plot_df["Yield_pred"].dropna(),
         marker="o", color="tab:green", label="XGBoost Predicted Yield (2021–2100)")
plt.xlabel("Year")
plt.ylabel("Mean yield across highly-harvested gridcells (>2000 hectares)")
plt.title("Wheat Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.xticks(range(1980, 2101, 10))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Combine historical + predicted into one series
combined_yield = pd.concat([
    plot_df.set_index("Year")["Yield_hist"].dropna(),
    plot_df.set_index("Year")["Yield_pred"].dropna()
])

plt.figure(figsize=(16, 6))
plt.plot(combined_yield.index, combined_yield.values, marker="o", color="tab:purple", label="Historical + Predicted Yield")
plt.xlabel("Year")
plt.ylabel("Mean yield (t/ha)")
plt.title("Wheat Yield across 1980–2100")
plt.grid(True, linestyle="--", alpha=0.6)
plt.xlim(1980, 2100)
plt.xticks(range(1980, 2101, 10))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance from XGBoost
importances = xgb_model.feature_importances_
features = numeric_features_wheat.columns
sorted_idx = np.argsort(importances)[::-1]
top_idx = sorted_idx[:10]

plt.figure(figsize=(10, 5))
sns.barplot(x=importances[top_idx], y=features[top_idx], ci=None)
plt.title("Feature Importance from XGBoost(Wheat)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
importances = xgb_model.feature_importances_
features = numeric_features_wheat.columns

# Create a DataFrame of features and their importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
})

# Sort by importance descending
importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Display top N (optional)
importance_df.head(10)

In [ ]:
# Prepare historical and predicted yield DataFrames
historical_xgb_df = historical_df[["Year", "Yield_hist"]].rename(columns={"Yield_hist": "Yield"})
historical_xgb_df = historical_xgb_df[(historical_xgb_df["Year"] >= 1980) & (historical_xgb_df["Year"] <= 2020)]

future_xgb_df = xgb_pred_df[xgb_pred_df["Year"] >= 2021]

# KDE plot for yield distribution
plt.figure(figsize=(8, 5))
sns.kdeplot(data=historical_xgb_df, x="Yield", label="1980–2020", fill=True)
sns.kdeplot(data=future_xgb_df, x="Yield", label="2021–2100", fill=True)
plt.title("Wheat Yield Distribution: Historical vs Predicted (XGBoost)")
plt.xlabel("Yield (t/ha)")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

## Unidirectional LSTM (Ran on Colab with Google drive)

In [ ]:
import numpy as np
import pandas as pd
import pyarrow
import os
from os import listdir
from os.path import isfile, join
from time import time
import re
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data import TensorDataset, DataLoader, random_split,DataLoader, Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ✅ Use correct path inside Drive
mypath = '/content/drive/MyDrive/the-future-crop-challenge'

In [ ]:
USE_CUDA = torch.cuda.is_available()
# DEVICE = torch.device('cuda:0')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load Data

In [ ]:
def get_region(row):
    if -125 <= row['lon'] <= -66 and 24 <= row['lat'] <= 50:
        return 'USA'
    elif -75 <= row['lon'] <= -35 and -40 <= row['lat'] <= -10:
        return 'South America'
    elif -10 <= row['lon'] <= 50 and 35 <= row['lat'] <= 60:
        return 'Europe'
    else:
        return 'Other'

In [ ]:
def load(crop,mode,data_dir):
    tasmax = pd.read_parquet(os.path.join(data_dir, "tasmax_{}_{}.parquet".format(crop,mode)))
    tasmin = pd.read_parquet(os.path.join(data_dir, "tasmin_{}_{}.parquet".format(crop,mode)))
    tas = pd.read_parquet(os.path.join(data_dir, "tas_{}_{}.parquet".format(crop,mode)))
    pr = pd.read_parquet(os.path.join(data_dir, "pr_{}_{}.parquet".format(crop,mode)))
    rsds = pd.read_parquet(os.path.join(data_dir, "rsds_{}_{}.parquet".format(crop,mode)))
    soil_co2 = pd.read_parquet(os.path.join(data_dir, "soil_co2_{}_{}.parquet".format(crop,mode)))

    if mode == 'train':
        yield_ = pd.read_parquet(os.path.join(data_dir, "{}_solutions_{}.parquet".format(mode,crop)))
        yield_ = yield_.values.astype(np.float32)

    if mode == 'test':
        yield_ = None

    climate = np.concatenate([
        tas.values[:, 5:,np.newaxis].astype(np.float32),
        tasmax.values[:, 5:,np.newaxis].astype(np.float32),
        tasmin.values[:, 5:,np.newaxis].astype(np.float32),
        pr.values[:, 5:,np.newaxis].astype(np.float32),
        rsds.values[:, 5:,np.newaxis].astype(np.float32),
    ], axis=2)
    return(climate,yield_,soil_co2)

In [ ]:
def get_training_dataloader(crop, mode, mypath, region, static_var, detrend=True):
    (climate, yield_label, soil) = load(crop, mode, mypath)

    # Reshape and calculate mean and std along grouped axis
    climate_mean = climate.reshape(climate.shape[0], 8, 30, 5).mean(axis=2)
    climate_sd = climate.reshape(climate.shape[0], 8, 30, 5).std(axis=2)

    GPP = np.maximum(climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,3] - 10, 0).sum(axis=2, keepdims=True).cumsum(axis=1)
    heat_stress_day = (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,1] > 30).sum(axis=2, keepdims=True).cumsum(axis=1)
    frost_days = (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,2] < 0).sum(axis=2, keepdims=True).cumsum(axis=1)

    climate_stats = np.concatenate([climate_sd, climate_mean, GPP, heat_stress_day, frost_days], axis=2)

    # Annual mean
    annual_df = pd.DataFrame(climate.mean(axis=1), columns=['tas', 'tasmax', 'tasmin', 'pr', 'rsds'])
    soil = soil.reset_index(drop=True)
    soil = pd.concat([annual_df, soil], axis=1)

    soil['region'] = soil.apply(get_region, axis=1)
    soil['co2'] = soil['co2'] / 1500

    # Detrend yield
    soil['yield'] = yield_label
    grouped_mean = soil.groupby(['lon', 'lat'])['yield'].mean().reset_index()
    grouped_mean = grouped_mean.rename(columns={'yield': 'yield_mean'})
    soil = pd.merge(soil, grouped_mean, on=['lon', 'lat'])
    soil['de_yield'] = soil['yield'] - soil['yield_mean']

    # Scale input features
    climate_arr = climate_stats.reshape(-1, climate_stats.shape[2])
    scaler = StandardScaler()
    scaler.fit(climate_arr)
    scaled_climate = scaler.transform(climate_arr).reshape(climate_stats.shape)

    # Scale static features
    scaler_static = StandardScaler()
    static_arr = soil[static_var].values.astype(np.float32)
    scaler_static.fit(static_arr)
    scaled_static = scaler_static.transform(static_arr)
    scaled_static = np.concatenate([scaled_static, soil['co2'].values.reshape(-1, 1)], axis=1)

    # Convert to torch tensors
    X = torch.from_numpy(scaled_climate).float()
    X_static = torch.from_numpy(scaled_static).float()
    y = torch.from_numpy(
        soil['de_yield'].values.reshape(-1, 1) if detrend else soil['yield'].values.reshape(-1, 1)
    ).float()

    dataset = TensorDataset(X, X_static, y)

    # Filter by region
    Region_index = soil[soil['region'].isin(region)].index.values
    dataset = torch.utils.data.Subset(dataset, Region_index)
    soil = soil.loc[Region_index].reset_index(drop=True)  # Align index with Subset

    print(f"After region filtering: {len(dataset)} samples")

    # 🟨 Year-based split using real_year
    if 'real_year' not in soil.columns:
        raise ValueError("'real_year' column not found in soil dataframe.")

    val_indices = soil[soil['real_year'] == 2020].index.tolist()
    train_indices = soil[soil['real_year'] != 2020].index.tolist()

    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    val_dataset = torch.utils.data.Subset(dataset, val_indices)

    # Optional: take last 10% of train as test set
    test_split = int(0.1 * len(train_dataset))
    test_dataset = torch.utils.data.Subset(train_dataset, list(range(len(train_dataset) - test_split, len(train_dataset))))
    train_dataset = torch.utils.data.Subset(train_dataset, list(range(0, len(train_dataset) - test_split)))

    # DataLoaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=3000, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=3000, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=3000, shuffle=False)

    print(f"Train: {len(train_loader.dataset)}, Val (2020): {len(val_loader.dataset)}, Test: {len(test_loader.dataset)}")
    return train_loader, val_loader, test_loader

### Train

In [ ]:
class LSTMForecaster_static(nn.Module):
  def __init__(self, n_features, n_hidden, n_outputs,n_static, n_static_hidden, sequence_len,DEVICE, n_lstm_layers=1, n_deep_layers=10, use_cuda=True, dropout=0.2):
    '''
    n_features: number of input features (1 for univariate forecasting)
    n_hidden: number of neurons in each hidden layer
    n_outputs: number of outputs to predict for each training example
    n_static: number of static features
    n_deep_layers: number of hidden dense layers after the lstm layer
    sequence_len: number of steps to look back at for prediction
    dropout: float (0 < dropout < 1) dropout ratio between dense layers
    '''
    super().__init__()
    self.n_lstm_layers = n_lstm_layers
    self.nhid = n_hidden
    self.use_cuda = use_cuda # set option for device selection
    self.DEVICE = DEVICE # set option for device selection

    # LSTM Layer
    self.lstm = nn.LSTM(n_features,
                        n_hidden,
                        num_layers=n_lstm_layers,
                        batch_first=True) # As we have transformed our data in this way


    # First dense layer to expand static variables
    self.fc1 = nn.Linear(n_static,n_static_hidden)
    self.relu1 = nn.ReLU()

    # Dropout layer
    self.dropout = nn.Dropout(p=dropout)
    # Create fully connected layers (n_hidden x n_deep_layers)
    dnn_layers = []

    # the first layer to concatenate hidden and static properties, 2 additional elements
    dnn_layers.append(nn.ReLU())
    dnn_layers.append(nn.Linear(n_hidden*n_lstm_layers+n_static_hidden, n_hidden))

    for i in range(n_deep_layers):
      # Last layer (n_hidden x n_outputs)
      if i == n_deep_layers - 1:
        dnn_layers.append(nn.ReLU())
        dnn_layers.append(nn.Linear(n_hidden, n_outputs))
      # All other layers (n_hidden x n_hidden) with dropout option
      else:
        dnn_layers.append(nn.ReLU())
        dnn_layers.append(nn.Linear(n_hidden, n_hidden))
        if dropout:
          dnn_layers.append(nn.Dropout(p=dropout))
    # compile DNN layers
    self.dnn = nn.Sequential(*dnn_layers)

  def forward(self, x,x_static):

    # Initialize hidden state
    hidden_state = torch.zeros(self.n_lstm_layers, x.shape[0], self.nhid)
    cell_state = torch.zeros(self.n_lstm_layers, x.shape[0], self.nhid)

    # move hidden state to device
    if self.use_cuda:
      hidden_state = hidden_state.to(self.DEVICE)
      cell_state = cell_state.to(self.DEVICE)

    self.hidden = (hidden_state, cell_state)
    # Forward Pass
    output, (h,c) = self.lstm(x, self.hidden) # LSTM

    # Flatten hidden state of the last step
    x_ = self.dropout(h.contiguous().view(x.shape[0], -1))
    x_static = self.relu1(self.fc1(x_static))
    # Pass forward hidden state and static features through fully connected DNN.
    return self.dnn(torch.cat((x_,x_static),axis=1)).squeeze()

In [ ]:
def train_model(model, train_loader, val_loader, test_loader,static_var,DEVICE,lr, weight_decay,step_size,sch_gamma,num_epochs=100):
    # Initialize the loss function and optimizer
    criterion = nn.MSELoss().to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=sch_gamma)
    #static & scaled
    # num_epochs = 100
    train_losses = [] #save performance metrics
    train_r2s = []
    val_losses = []
    val_r2s = []
    model.train()
    for epoch in range(num_epochs):
        start = time()
        model.train()
        for i, (x, x_static, y_true), in enumerate(train_loader):
            # model.train()
            # print(i)
            xt,x_st, yt = x.to(DEVICE),x_static.to(DEVICE),y_true.to(DEVICE).squeeze()

            y_pred = model(xt,x_st)
            loss_batch = criterion(y_pred, yt)
            optimizer.zero_grad()
            loss_batch.backward()
            optimizer.step()
        scheduler.step()

        if epoch % 10 != 0:
            continue
        train_loss,train_r2 = test_mse_r2_static(model, criterion, train_loader,DEVICE)
        val_loss,val_r2 = test_mse_r2_static(model, criterion, val_loader,DEVICE)
        test_loss,test_r2 = test_mse_r2_static(model, criterion, test_loader,DEVICE)

        train_losses.append(train_loss)
        train_r2s.append(train_r2)
        val_losses.append(val_loss)
        val_r2s.append(val_r2)
        end = time()

        print("Epoch %d: TRAIN MSE loss: %.6f, TRAIN r2 loss: %.6f" % (epoch, train_loss,train_r2))
        print("Epoch %d: VAL MSE loss: %.6f, VAL r2: %.6f" % (epoch, val_loss, val_r2))
        print("Epoch %d: TEST MSE loss: %.6f, TEST r2: %.6f" % (epoch, test_loss, test_r2))
        print("Finish with for one epoch:{} second".format(end - start))
    return model

In [ ]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
def test_mse_r2_static(model,loss_fn, dataloader,DEVICE):
    model.eval()
    loss_test_cum = 0
    y_test_pred = []
    y_test_label = []
    with torch.no_grad():
        for i, (x, z, y) in enumerate(dataloader):
            y_pred = model(x.to(DEVICE),z.to(DEVICE))
            # predicted = model(input)
            loss   = loss_fn(y_pred, y.to(DEVICE).squeeze())
            loss_test_cum += loss
            y_test_pred.append(y_pred)
            y_test_label.append(y.to(DEVICE).squeeze())
        r2 = r2_loss(torch.cat(y_test_pred),torch.cat(y_test_label))
    return loss_test_cum/(i+1),r2

In [ ]:
# (climate, yield_label, soil) = load('maize','train',mypath)
static_var=['nitrogen','texture_class','lon','lat','yield_mean','tas', 'tasmax', 'tasmin', 'pr', 'rsds']

### Maize

In [ ]:
## make maize train_loader
train_loader,val_loader,test_loader = get_training_dataloader('maize','train',mypath,region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
(x,z,y) = next(iter(train_loader))

In [ ]:
# global model training
model = LSTMForecaster_static(n_features = x.shape[2], n_hidden = 200, n_outputs = 1,
                              n_static = len(static_var)+1,n_static_hidden=200,sequence_len = x.shape[1],
                              DEVICE = DEVICE, n_deep_layers=4, use_cuda=USE_CUDA, dropout=0.4).to(DEVICE)

maize_unscale_model_GLOB = train_model(model, train_loader,val_loader,test_loader,static_var,DEVICE,
                                  lr=4e-4, weight_decay=0.1,step_size=100,sch_gamma=0.8,num_epochs=200)

### Wheat

In [ ]:
## make wheat train_loader
train_loader,val_loader,test_loader = get_training_dataloader('wheat','train',mypath,
                                                              region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
(x,z,y) = next(iter(train_loader))

In [ ]:
# # global model training
model = LSTMForecaster_static(
    n_features = x.shape[2],
    n_hidden = 128,
    n_outputs = 1,
    n_static = len(static_var) + 1,
    n_static_hidden = 128,
    sequence_len = x.shape[1],
    DEVICE = DEVICE,
    n_lstm_layers = 1,
    n_deep_layers = 3,
    use_cuda = USE_CUDA,
    dropout = 0.4
).to(DEVICE)

wheat_unscale_model_GLOB = train_model(model, train_loader,val_loader,test_loader,static_var,DEVICE,
                                   lr=4e-4, weight_decay=0.01,step_size=50,sch_gamma=0.8,num_epochs=200)

### Evalutation

In [ ]:
def get_test_dataloader(climate,climate_test,soil,soil_test,region,static_var):
    # Reshape and calculate mean and sd along the grouped axis
    climate_test_mean = climate_test.reshape(climate_test.shape[0], 8, 30, 5).mean(axis=2)
    climate_test_sd = climate_test.reshape(climate_test.shape[0], 8, 30, 5).std(axis=2)
    GPP_test = np.maximum(climate_test.reshape(climate_test.shape[0], 8, 30, 5)[:,:,:,3] - 10, 0).sum(axis=2,keepdims=True).cumsum(axis=1) # cumulative GDD by phases
    heat_stress_day_test =  (climate_test.reshape(climate_test.shape[0], 8, 30, 5)[:,:,:,1] > 30).sum(axis=2,keepdims=True).cumsum(axis=1)
    frost_days_test = (climate_test.reshape(climate_test.shape[0], 8, 30, 5)[:,:,:,2] < 0).sum(axis=2,keepdims=True).cumsum(axis=1)
    climate_test_stats = np.concatenate([climate_test_sd,climate_test_mean,GPP_test,heat_stress_day_test,frost_days_test],axis=2)

    climate_mean = climate.reshape(climate.shape[0], 8, 30, 5).mean(axis=2)
    climate_sd = climate.reshape(climate.shape[0], 8, 30, 5).std(axis=2)
    GPP = np.maximum(climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,3] - 10, 0).sum(axis=2,keepdims=True).cumsum(axis=1) # cumulative GDD by phases
    heat_stress_day =  (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,1] > 30).sum(axis=2,keepdims=True).cumsum(axis=1)
    frost_days = (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,2] < 0).sum(axis=2,keepdims=True).cumsum(axis=1)
    climate_stats = np.concatenate([climate_sd,climate_mean,GPP,heat_stress_day,frost_days],axis=2)

    #annual mean for test
    annual_test_df = pd.DataFrame(climate_test.mean(axis=1),columns=['tas', 'tasmax', 'tasmin', 'pr', 'rsds'])
    soil_test = pd.concat([annual_test_df, soil_test], axis=1)

    #annual mean for train
    annual_df = pd.DataFrame(climate.mean(axis=1),columns=['tas', 'tasmax', 'tasmin', 'pr', 'rsds'])
    soil = pd.concat([annual_df, soil], axis=1)

    #scale input features
    climate_arr = climate_stats.reshape(-1, climate_stats.shape[2])
    scaler = StandardScaler()
    scaler.fit(climate_arr)
    # scaled_climate = scaler.transform(climate_arr).reshape(climate_stats.shape)

    #scale static features
    scaler_static = StandardScaler()
    static_arr = soil[static_var].values.astype(np.float32)
    scaler_static.fit(static_arr)
    # scaler_static.fit(static_arr)

    climate_test_arr = climate_test_stats.reshape(-1, climate_test_stats.shape[2])
    climate_test_scale = scaler.transform(climate_test_arr).reshape(climate_test_stats.shape)

    static_test_arr = soil_test[static_var].values.astype(np.float32)
    static_test = scaler_static.transform(static_test_arr)
    #add co2
    soil_test['co2'] = soil_test['co2']/1500
    static_test = np.concatenate([static_test,soil_test['co2'].values.reshape(-1, 1)],axis = 1)

    X_test_to_loader = torch.from_numpy(climate_test_scale).type(torch.FloatTensor)
    static_test_loader = torch.from_numpy(static_test).type(torch.FloatTensor)

    dataset = TensorDataset(X_test_to_loader, static_test_loader)

    # subset by REGION
    Region_index = soil_test[soil_test['region'].isin(region)].index.values
    dataset = torch.utils.data.Subset(dataset, Region_index)
    print(len(dataset))

    test_loader_stats = torch.utils.data.DataLoader(dataset, 2000, shuffle=False)

    return soil_test, Region_index, test_loader_stats

In [ ]:
## make maize test_loader
crop = 'maize'
(climate, yield_label, soil) = load(crop,'train', mypath)
(climate_test, yield_, soil_test_maize) = load(crop,'test', mypath)

In [ ]:
#detrend yield
soil = soil.reset_index()
soil['yield'] = yield_label
grouped_mean = soil.groupby(['lon', 'lat'])['yield'].mean().reset_index()
grouped_mean = grouped_mean.rename(columns={'yield': 'yield_mean'})
soil = pd.merge(soil, grouped_mean, on=['lon', 'lat'])

soil_test_maize = soil_test_maize.reset_index()
soil_test_maize = pd.merge(soil_test_maize, grouped_mean, on=['lon', 'lat'])

soil['de_yield'] = soil['yield']-soil['yield_mean']
soil['region'] = soil.apply(get_region, axis=1)
soil_test_maize['region'] = soil_test_maize.apply(get_region, axis=1)

In [ ]:
test_soil, Region_index_US, test_loader_US = get_test_dataloader(climate,climate_test,soil,soil_test_maize,region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
maize_pred_stats_test = []
for i, (x,z) in enumerate(test_loader_US):
    with torch.no_grad():
        # y_pred = maize_unscale_model_US(x.to(DEVICE),z.to(DEVICE))
        y_pred = maize_unscale_model_GLOB(x.to(DEVICE),z.to(DEVICE))
        maize_pred_stats_test.append(y_pred.detach().cpu().numpy())

In [ ]:
## make wheat test_loader
crop = 'wheat'
(climate, yield_label, soil) = load(crop,'train', mypath)
(climate_test, yield_, soil_test_wheat) = load(crop,'test', mypath)

In [ ]:
#detrend yield
soil = soil.reset_index()
soil['yield'] = yield_label
grouped_mean = soil.groupby(['lon', 'lat'])['yield'].mean().reset_index()
grouped_mean = grouped_mean.rename(columns={'yield': 'yield_mean'})
soil = pd.merge(soil, grouped_mean, on=['lon', 'lat'])

soil_test_wheat = soil_test_wheat.reset_index()
soil_test_wheat = pd.merge(soil_test_wheat, grouped_mean, on=['lon', 'lat'])

soil['de_yield'] = soil['yield']-soil['yield_mean']
soil['region'] = soil.apply(get_region, axis=1)
soil_test_wheat['region'] = soil_test_wheat.apply(get_region, axis=1)

In [ ]:
test_soil, Region_index_US, test_loader_US = get_test_dataloader(climate,climate_test,soil,soil_test_wheat,region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
wheat_pred_stats_test = []
for i, (x,z) in enumerate(test_loader_US):
    with torch.no_grad():
        # y_pred = maize_unscale_model_US(x.to(DEVICE),z.to(DEVICE))
        y_pred = wheat_unscale_model_GLOB(x.to(DEVICE),z.to(DEVICE))
        wheat_pred_stats_test.append(y_pred.detach().cpu().numpy())

We skipped the submission part as that was an attemp for Kaggle submission

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.hist(soil_test_maize['preds_yield'], bins=50, alpha=0.6, label='Maize')
plt.hist(soil_test_wheat['preds_yield'], bins=50, alpha=0.6, label='Wheat')
plt.xlabel('Predicted Yield')
plt.ylabel('Number of Grid Cells')
plt.title('Distribution of Predicted Yields (Test Set)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(soil_test_maize['lon'], soil_test_maize['lat'],
            c=soil_test_maize['preds_yield'],  s=20, alpha=0.7)
plt.colorbar(label='Predicted Maize Yield')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Distribution of Predicted Maize Yields')
plt.show()

In [ ]:
static_var=['nitrogen','texture_class','lon','lat','yield_mean','tas', 'tasmax', 'tasmin', 'pr', 'rsds']

# For Maize
_, _, maize_test_loader = get_training_dataloader(
    'maize', 'train', mypath,
    region=['USA', 'Other', 'South America', 'Europe'],
    static_var=static_var
)

# For Wheat
_, _, wheat_test_loader = get_training_dataloader(
    'wheat', 'train', mypath,
    region=['USA', 'Other', 'South America', 'Europe'],
    static_var=static_var
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

def evaluate_lstm(model, test_loader, title_prefix=""):
    model.eval()
    y_true_all, y_pred_all = [], []

    with torch.no_grad():
        for x, x_static, y_true in test_loader:
            y_pred = model(x.to(DEVICE), x_static.to(DEVICE)).cpu().numpy()
            y_true = y_true.cpu().numpy()
            y_true_all.append(y_true)
            y_pred_all.append(y_pred)

    y_true_all = np.concatenate(y_true_all)
    y_pred_all = np.concatenate(y_pred_all)
    residuals = y_true_all - y_pred_all
    r2 = r2_score(y_true_all, y_pred_all)
    rmse = np.sqrt(mean_squared_error(y_true_all, y_pred_all))

    return y_true_all, y_pred_all, residuals, r2, rmse

# --- Evaluate both models on 2020 validation set ---
y_true_wheat, y_pred_wheat, resid_wheat, r2_wheat, rmse_wheat = evaluate_lstm(wheat_unscale_model_GLOB, val_loader, "Wheat (2020)")
y_true_maize, y_pred_maize, resid_maize, r2_maize, rmse_maize = evaluate_lstm(maize_unscale_model_GLOB, val_loader, "Maize (2020)")

# --- Plotting ---
fig, axs = plt.subplots(2, 2, figsize=(14, 8))

# 1. Wheat: Pred vs Actual
axs[0, 0].scatter(y_true_wheat, y_pred_wheat, alpha=0.5, s=15, edgecolor='k')
axs[0, 0].plot([y_true_wheat.min(), y_true_wheat.max()],
               [y_true_wheat.min(), y_true_wheat.max()],
               'r--', label='Perfect Prediction')
axs[0, 0].set_title(f'Validation (2020) - Wheat\nR² = {r2_wheat:.4f}, RMSE = {rmse_wheat:.3f}')
axs[0, 0].set_xlabel("Actual Yield")
axs[0, 0].set_ylabel("Predicted Yield")
axs[0, 0].legend()
axs[0, 0].grid(True)

# 2. Maize: Pred vs Actual
axs[0, 1].scatter(y_true_maize, y_pred_maize, alpha=0.5, s=15, edgecolor='k')
axs[0, 1].plot([y_true_maize.min(), y_true_maize.max()],
               [y_true_maize.min(), y_true_maize.max()],
               'r--', label='Perfect Prediction')
axs[0, 1].set_title(f'Validation (2020) - Maize\nR² = {r2_maize:.4f}, RMSE = {rmse_maize:.3f}')
axs[0, 1].set_xlabel("Actual Yield")
axs[0, 1].set_ylabel("Predicted Yield")
axs[0, 1].legend()
axs[0, 1].grid(True)

# 3. Residuals Histogram - Wheat
resid_w = resid_wheat.ravel()
mean_w, std_w = np.mean(resid_w), np.std(resid_w)
x_w = np.linspace(resid_w.min(), resid_w.max(), 500)
pdf_w = (1 / (std_w * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_w - mean_w) / std_w) ** 2)
axs[1, 0].hist(resid_w, bins=50, color='forestgreen', edgecolor='black', alpha=0.7, density=True)
axs[1, 0].plot(x_w, pdf_w, 'k--')
axs[1, 0].set_title('Residuals Histogram (Wheat - 2020)')
axs[1, 0].set_xlabel("Residual")
axs[1, 0].set_ylabel("Density")
axs[1, 0].grid(True)

# 4. Residuals Histogram - Maize
resid_m = resid_maize.ravel()
mean_m, std_m = np.mean(resid_m), np.std(resid_m)
x_m = np.linspace(resid_m.min(), resid_m.max(), 500)
pdf_m = (1 / (std_m * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_m - mean_m) / std_m) ** 2)
axs[1, 1].hist(resid_m, bins=50, color='darkgreen', edgecolor='black', alpha=0.7, density=True)
axs[1, 1].plot(x_m, pdf_m, 'k--')
axs[1, 1].set_title('Residuals Histogram (Maize - 2020)')
axs[1, 1].set_xlabel("Residual")
axs[1, 1].set_ylabel("Density")
axs[1, 1].grid(True)

plt.tight_layout()
plt.show()

## Bidirectional LSTM

In [ ]:
import numpy as np
import pandas as pd
import pyarrow
import os
from os import listdir
from os.path import isfile, join
from time import time
import re
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data import TensorDataset, DataLoader, random_split,DataLoader, Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
mypath = "/content/drive/MyDrive/the-future-crop-challenge"

In [ ]:
USE_CUDA = torch.cuda.is_available()
# DEVICE = torch.device('cuda:0')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_region(row):
    if -125 <= row['lon'] <= -66 and 24 <= row['lat'] <= 50:
        return 'USA'
    elif -75 <= row['lon'] <= -35 and -40 <= row['lat'] <= -10:
        return 'South America'
    elif -10 <= row['lon'] <= 50 and 35 <= row['lat'] <= 60:
        return 'Europe'
    else:
        return 'Other'

In [ ]:
def load(crop,mode,data_dir):
    tasmax = pd.read_parquet(os.path.join(data_dir, "tasmax_{}_{}.parquet".format(crop,mode)))
    tasmin = pd.read_parquet(os.path.join(data_dir, "tasmin_{}_{}.parquet".format(crop,mode)))
    tas = pd.read_parquet(os.path.join(data_dir, "tas_{}_{}.parquet".format(crop,mode)))
    pr = pd.read_parquet(os.path.join(data_dir, "pr_{}_{}.parquet".format(crop,mode)))
    rsds = pd.read_parquet(os.path.join(data_dir, "rsds_{}_{}.parquet".format(crop,mode)))
    soil_co2 = pd.read_parquet(os.path.join(data_dir, "soil_co2_{}_{}.parquet".format(crop,mode)))

    if mode == 'train':
        yield_ = pd.read_parquet(os.path.join(data_dir, "{}_solutions_{}.parquet".format(mode,crop)))
        yield_ = yield_.values.astype(np.float32)

    if mode == 'test':
        yield_ = None

    climate = np.concatenate([
        tas.values[:, 5:,np.newaxis].astype(np.float32),
        tasmax.values[:, 5:,np.newaxis].astype(np.float32),
        tasmin.values[:, 5:,np.newaxis].astype(np.float32),
        pr.values[:, 5:,np.newaxis].astype(np.float32),
        rsds.values[:, 5:,np.newaxis].astype(np.float32),
    ], axis=2)
    return(climate,yield_,soil_co2)

In [ ]:
def get_training_dataloader(crop, mode, mypath, region, static_var, detrend=True):
    (climate, yield_label, soil) = load(crop, mode, mypath)

    # Reshape and calculate mean and std along grouped axis
    climate_mean = climate.reshape(climate.shape[0], 8, 30, 5).mean(axis=2)
    climate_sd = climate.reshape(climate.shape[0], 8, 30, 5).std(axis=2)

    GPP = np.maximum(climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,3] - 10, 0).sum(axis=2, keepdims=True).cumsum(axis=1)
    heat_stress_day = (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,1] > 30).sum(axis=2, keepdims=True).cumsum(axis=1)
    frost_days = (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,2] < 0).sum(axis=2, keepdims=True).cumsum(axis=1)

    climate_stats = np.concatenate([climate_sd, climate_mean, GPP, heat_stress_day, frost_days], axis=2)

    # Annual mean
    annual_df = pd.DataFrame(climate.mean(axis=1), columns=['tas', 'tasmax', 'tasmin', 'pr', 'rsds'])
    soil = soil.reset_index(drop=True)
    soil = pd.concat([annual_df, soil], axis=1)

    soil['region'] = soil.apply(get_region, axis=1)
    soil['co2'] = soil['co2'] / 1500

    # Detrend yield
    soil['yield'] = yield_label
    grouped_mean = soil.groupby(['lon', 'lat'])['yield'].mean().reset_index()
    grouped_mean = grouped_mean.rename(columns={'yield': 'yield_mean'})
    soil = pd.merge(soil, grouped_mean, on=['lon', 'lat'])
    soil['de_yield'] = soil['yield'] - soil['yield_mean']

    # Scale input features
    climate_arr = climate_stats.reshape(-1, climate_stats.shape[2])
    scaler = StandardScaler()
    scaler.fit(climate_arr)
    scaled_climate = scaler.transform(climate_arr).reshape(climate_stats.shape)

    # Scale static features
    scaler_static = StandardScaler()
    static_arr = soil[static_var].values.astype(np.float32)
    scaler_static.fit(static_arr)
    scaled_static = scaler_static.transform(static_arr)
    scaled_static = np.concatenate([scaled_static, soil['co2'].values.reshape(-1, 1)], axis=1)

    # Convert to torch tensors
    X = torch.from_numpy(scaled_climate).float()
    X_static = torch.from_numpy(scaled_static).float()
    y = torch.from_numpy(
        soil['de_yield'].values.reshape(-1, 1) if detrend else soil['yield'].values.reshape(-1, 1)
    ).float()

    dataset = TensorDataset(X, X_static, y)

    # Filter by region
    Region_index = soil[soil['region'].isin(region)].index.values
    dataset = torch.utils.data.Subset(dataset, Region_index)
    soil = soil.loc[Region_index].reset_index(drop=True)  # Align index with Subset

    print(f"After region filtering: {len(dataset)} samples")

    # 🟨 Year-based split using real_year
    if 'real_year' not in soil.columns:
        raise ValueError("'real_year' column not found in soil dataframe.")

    val_indices = soil[soil['real_year'] == 2020].index.tolist()
    train_indices = soil[soil['real_year'] != 2020].index.tolist()

    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    val_dataset = torch.utils.data.Subset(dataset, val_indices)

    # Optional: take last 10% of train as test set
    test_split = int(0.1 * len(train_dataset))
    test_dataset = torch.utils.data.Subset(train_dataset, list(range(len(train_dataset) - test_split, len(train_dataset))))
    train_dataset = torch.utils.data.Subset(train_dataset, list(range(0, len(train_dataset) - test_split)))

    # DataLoaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=3000, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=3000, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=3000, shuffle=False)

    print(f"Train: {len(train_loader.dataset)}, Val (2020): {len(val_loader.dataset)}, Test: {len(test_loader.dataset)}")
    return train_loader, val_loader, test_loader

In [ ]:
# Updated BiLSTMForecaster_static using custom BiLSTM from notebook.py
import torch
import torch.nn as nn

# ---- Custom LSTM definitions ----
class CustomLSTM(nn.Module):
    class _LSTM(nn.Module):
        def __init__(self, input_size, hidden_size):
            super().__init__()
            self.hidden_size = hidden_size
            self.xh2gate = nn.Linear(input_size + hidden_size, 4 * hidden_size)

        def forward(self, x_t, h_t, c_t):
            combined = torch.cat((x_t, h_t), dim=1)
            gates = self.xh2gate(combined)
            i, f, o, g = gates.chunk(4, dim=1)
            i = torch.sigmoid(i)
            f = torch.sigmoid(f)
            o = torch.sigmoid(o)
            g = torch.tanh(g)
            c_t = f * c_t + i * g
            h_t = o * torch.tanh(c_t)
            return h_t, c_t

    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.layers = nn.ModuleList([
            self._LSTM(input_size if i == 0 else hidden_size, hidden_size)
            for i in range(num_layers)
        ])

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        h = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
        c = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
        for t in range(seq_len):
            input_t = x[:, t, :]
            for layer in range(self.num_layers):
                h[layer], c[layer] = self.layers[layer](input_t, h[layer], c[layer])
                input_t = h[layer]
        return h[-1]

class CustomBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.forward_lstm = CustomLSTM(input_size, hidden_size, num_layers)
        self.backward_lstm = CustomLSTM(input_size, hidden_size, num_layers)

    def forward(self, x):
        out_fwd = self.forward_lstm(x)
        x_rev = torch.flip(x, dims=[1])
        out_bwd = self.backward_lstm(x_rev)
        return torch.cat([out_fwd, out_bwd], dim=1)

# ---- Modified Forecaster using Custom BiLSTM ----
class LSTMForecaster_static(nn.Module):
    def __init__(self, n_features, n_hidden, n_outputs, n_static, n_static_hidden,
                 sequence_len, DEVICE, n_lstm_layers=1, n_deep_layers=10, use_cuda=True, dropout=0.2):
        super().__init__()
        self.nhid = n_hidden
        self.DEVICE = DEVICE
        self.use_cuda = use_cuda

        self.bilstm = CustomBiLSTM(n_features, n_hidden, n_lstm_layers)

        self.fc1 = nn.Linear(n_static, n_static_hidden)
        self.relu1 = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

        dnn_layers = [nn.ReLU(),
                      nn.Linear(2 * n_hidden + n_static_hidden, n_hidden)]
        for i in range(n_deep_layers):
            dnn_layers.append(nn.ReLU())
            if i == n_deep_layers - 1:
                dnn_layers.append(nn.Linear(n_hidden, n_outputs))
            else:
                dnn_layers.append(nn.Linear(n_hidden, n_hidden))
                dnn_layers.append(nn.Dropout(p=dropout))

        self.dnn = nn.Sequential(*dnn_layers)

    def forward(self, x, x_static):
        bilstm_out = self.dropout(self.bilstm(x))
        x_static = self.relu1(self.fc1(x_static))
        return self.dnn(torch.cat((bilstm_out, x_static), dim=1)).squeeze()

In [ ]:
def train_model(model, train_loader, val_loader, test_loader,static_var,DEVICE,lr, weight_decay,step_size,sch_gamma,num_epochs=100):
    # Initialize the loss function and optimizer
    criterion = nn.MSELoss().to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=sch_gamma)
    #static & scaled
    # num_epochs = 100
    train_losses = [] #save performance metrics
    train_r2s = []
    val_losses = []
    val_r2s = []
    model.train()
    for epoch in range(num_epochs):
        start = time()
        model.train()
        for i, (x, x_static, y_true), in enumerate(train_loader):
            # model.train()
            # print(i)
            xt,x_st, yt = x.to(DEVICE),x_static.to(DEVICE),y_true.to(DEVICE).squeeze()

            y_pred = model(xt,x_st)
            loss_batch = criterion(y_pred, yt)
            optimizer.zero_grad()
            loss_batch.backward()
            optimizer.step()
        scheduler.step()

        if epoch % 10 != 0:
            continue
        train_loss,train_r2 = test_mse_r2_static(model, criterion, train_loader,DEVICE)
        val_loss,val_r2 = test_mse_r2_static(model, criterion, val_loader,DEVICE)
        test_loss,test_r2 = test_mse_r2_static(model, criterion, test_loader,DEVICE)

        train_losses.append(train_loss)
        train_r2s.append(train_r2)
        val_losses.append(val_loss)
        val_r2s.append(val_r2)
        end = time()

        print("Epoch %d: TRAIN MSE loss: %.6f, TRAIN r2 loss: %.6f" % (epoch, train_loss,train_r2))
        print("Epoch %d: VAL MSE loss: %.6f, VAL r2: %.6f" % (epoch, val_loss, val_r2))
        print("Epoch %d: TEST MSE loss: %.6f, TEST r2: %.6f" % (epoch, test_loss, test_r2))
        print("Finish with for one epoch:{} second".format(end - start))
    return model

In [ ]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
def test_mse_r2_static(model,loss_fn, dataloader,DEVICE):
    model.eval()
    loss_test_cum = 0
    y_test_pred = []
    y_test_label = []
    with torch.no_grad():
        for i, (x, z, y) in enumerate(dataloader):
            y_pred = model(x.to(DEVICE),z.to(DEVICE))
            # predicted = model(input)
            loss   = loss_fn(y_pred, y.to(DEVICE).squeeze())
            loss_test_cum += loss
            y_test_pred.append(y_pred)
            y_test_label.append(y.to(DEVICE).squeeze())
        r2 = r2_loss(torch.cat(y_test_pred),torch.cat(y_test_label))
    return loss_test_cum/(i+1),r2

In [ ]:
# (climate, yield_label, soil) = load('maize','train',mypath)
static_var=['nitrogen','texture_class','lon','lat','yield_mean','tas', 'tasmax', 'tasmin', 'pr', 'rsds']

In [ ]:
## make maize train_loader
train_loader,val_loader,test_loader = get_training_dataloader('maize','train',mypath,region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
(x,z,y) = next(iter(train_loader))

In [ ]:
# global model training
model = LSTMForecaster_static(n_features = x.shape[2], n_hidden = 200, n_outputs = 1,
                              n_static = len(static_var)+1,n_static_hidden=200,sequence_len = x.shape[1],
                              DEVICE = DEVICE, n_deep_layers=4, use_cuda=USE_CUDA, dropout=0.4).to(DEVICE)

maize_unscale_model_GLOB = train_model(model, train_loader,val_loader,test_loader,static_var,DEVICE,
                                  lr=4e-4, weight_decay=0.1,step_size=100,sch_gamma=0.8,num_epochs=200)

In [ ]:
## make wheat train_loader
train_loader,val_loader,test_loader = get_training_dataloader('wheat','train',mypath,
                                                              region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
(x,z,y) = next(iter(train_loader))

In [ ]:
# global model training
model = LSTMForecaster_static(n_features = x.shape[2], n_hidden = 200, n_outputs = 1,
                              n_static = len(static_var)+1,n_static_hidden=200,sequence_len = x.shape[1],
                              DEVICE = DEVICE, n_deep_layers=4, use_cuda=USE_CUDA, dropout=0.4).to(DEVICE)

wheat_unscale_model_GLOB2 = train_model(model, train_loader,val_loader,test_loader,static_var,DEVICE,
                                  lr=4e-4, weight_decay=0.1,step_size=100,sch_gamma=0.8,num_epochs=200)

In [ ]:
def get_test_dataloader(climate,climate_test,soil,soil_test,region,static_var):
    # Reshape and calculate mean and sd along the grouped axis
    climate_test_mean = climate_test.reshape(climate_test.shape[0], 8, 30, 5).mean(axis=2)
    climate_test_sd = climate_test.reshape(climate_test.shape[0], 8, 30, 5).std(axis=2)
    GPP_test = np.maximum(climate_test.reshape(climate_test.shape[0], 8, 30, 5)[:,:,:,3] - 10, 0).sum(axis=2,keepdims=True).cumsum(axis=1) # cumulative GDD by phases
    heat_stress_day_test =  (climate_test.reshape(climate_test.shape[0], 8, 30, 5)[:,:,:,1] > 30).sum(axis=2,keepdims=True).cumsum(axis=1)
    frost_days_test = (climate_test.reshape(climate_test.shape[0], 8, 30, 5)[:,:,:,2] < 0).sum(axis=2,keepdims=True).cumsum(axis=1)
    climate_test_stats = np.concatenate([climate_test_sd,climate_test_mean,GPP_test,heat_stress_day_test,frost_days_test],axis=2)

    climate_mean = climate.reshape(climate.shape[0], 8, 30, 5).mean(axis=2)
    climate_sd = climate.reshape(climate.shape[0], 8, 30, 5).std(axis=2)
    GPP = np.maximum(climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,3] - 10, 0).sum(axis=2,keepdims=True).cumsum(axis=1) # cumulative GDD by phases
    heat_stress_day =  (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,1] > 30).sum(axis=2,keepdims=True).cumsum(axis=1)
    frost_days = (climate.reshape(climate.shape[0], 8, 30, 5)[:,:,:,2] < 0).sum(axis=2,keepdims=True).cumsum(axis=1)
    climate_stats = np.concatenate([climate_sd,climate_mean,GPP,heat_stress_day,frost_days],axis=2)

    #annual mean for test
    annual_test_df = pd.DataFrame(climate_test.mean(axis=1),columns=['tas', 'tasmax', 'tasmin', 'pr', 'rsds'])
    soil_test = pd.concat([annual_test_df, soil_test], axis=1)

    #annual mean for train
    annual_df = pd.DataFrame(climate.mean(axis=1),columns=['tas', 'tasmax', 'tasmin', 'pr', 'rsds'])
    soil = pd.concat([annual_df, soil], axis=1)

    #scale input features
    climate_arr = climate_stats.reshape(-1, climate_stats.shape[2])
    scaler = StandardScaler()
    scaler.fit(climate_arr)
    # scaled_climate = scaler.transform(climate_arr).reshape(climate_stats.shape)

    #scale static features
    scaler_static = StandardScaler()
    static_arr = soil[static_var].values.astype(np.float32)
    scaler_static.fit(static_arr)
    # scaler_static.fit(static_arr)

    climate_test_arr = climate_test_stats.reshape(-1, climate_test_stats.shape[2])
    climate_test_scale = scaler.transform(climate_test_arr).reshape(climate_test_stats.shape)

    static_test_arr = soil_test[static_var].values.astype(np.float32)
    static_test = scaler_static.transform(static_test_arr)
    #add co2
    soil_test['co2'] = soil_test['co2']/1500
    static_test = np.concatenate([static_test,soil_test['co2'].values.reshape(-1, 1)],axis = 1)

    X_test_to_loader = torch.from_numpy(climate_test_scale).type(torch.FloatTensor)
    static_test_loader = torch.from_numpy(static_test).type(torch.FloatTensor)

    dataset = TensorDataset(X_test_to_loader, static_test_loader)

    # subset by REGION
    Region_index = soil_test[soil_test['region'].isin(region)].index.values
    dataset = torch.utils.data.Subset(dataset, Region_index)
    print(len(dataset))

    test_loader_stats = torch.utils.data.DataLoader(dataset, 2000, shuffle=False)

    return soil_test, Region_index, test_loader_stats

In [ ]:
## make maize test_loader
crop = 'maize'
(climate, yield_label, soil) = load(crop,'train', mypath)
(climate_test, yield_, soil_test_maize) = load(crop,'test', mypath)

In [ ]:
#detrend yield
soil = soil.reset_index()
soil['yield'] = yield_label
grouped_mean = soil.groupby(['lon', 'lat'])['yield'].mean().reset_index()
grouped_mean = grouped_mean.rename(columns={'yield': 'yield_mean'})
soil = pd.merge(soil, grouped_mean, on=['lon', 'lat'])

soil_test_maize = soil_test_maize.reset_index()
soil_test_maize = pd.merge(soil_test_maize, grouped_mean, on=['lon', 'lat'])

soil['de_yield'] = soil['yield']-soil['yield_mean']
soil['region'] = soil.apply(get_region, axis=1)
soil_test_maize['region'] = soil_test_maize.apply(get_region, axis=1)

In [ ]:
test_soil, Region_index_US, test_loader_US = get_test_dataloader(climate,climate_test,soil,soil_test_maize,region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
maize_pred_stats_test = []
for i, (x,z) in enumerate(test_loader_US):
    with torch.no_grad():
        # y_pred = maize_unscale_model_US(x.to(DEVICE),z.to(DEVICE))
        y_pred = maize_unscale_model_GLOB(x.to(DEVICE),z.to(DEVICE))
        maize_pred_stats_test.append(y_pred.detach().cpu().numpy())

In [ ]:
## make wheat test_loader
crop = 'wheat'
(climate, yield_label, soil) = load(crop,'train', mypath)
(climate_test, yield_, soil_test_wheat) = load(crop,'test', mypath)

In [ ]:
#detrend yield
soil = soil.reset_index()
soil['yield'] = yield_label
grouped_mean = soil.groupby(['lon', 'lat'])['yield'].mean().reset_index()
grouped_mean = grouped_mean.rename(columns={'yield': 'yield_mean'})
soil = pd.merge(soil, grouped_mean, on=['lon', 'lat'])

soil_test_wheat = soil_test_wheat.reset_index()
soil_test_wheat = pd.merge(soil_test_wheat, grouped_mean, on=['lon', 'lat'])

soil['de_yield'] = soil['yield']-soil['yield_mean']
soil['region'] = soil.apply(get_region, axis=1)
soil_test_wheat['region'] = soil_test_wheat.apply(get_region, axis=1)

In [ ]:
test_soil, Region_index_US, test_loader_US = get_test_dataloader(climate,climate_test,soil,soil_test_wheat,region=['USA', 'Other', 'South America', 'Europe'],static_var=static_var)

In [ ]:
wheat_pred_stats_test = []
for i, (x,z) in enumerate(test_loader_US):
    with torch.no_grad():
        # y_pred = maize_unscale_model_US(x.to(DEVICE),z.to(DEVICE))
        y_pred = wheat_unscale_model_GLOB2(x.to(DEVICE),z.to(DEVICE))
        wheat_pred_stats_test.append(y_pred.detach().cpu().numpy())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.hist(soil_test_maize['preds_yield'], bins=50, alpha=0.6, label='Maize')
plt.hist(soil_test_wheat['preds_yield'], bins=50, alpha=0.6, label='Wheat')
plt.xlabel('Predicted Yield')
plt.ylabel('Number of Grid Cells')
plt.title('Distribution of Predicted Yields (Test Set)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(soil_test_maize['lon'], soil_test_maize['lat'],
            c=soil_test_maize['preds_yield'],  s=20, alpha=0.7)
plt.colorbar(label='Predicted Maize Yield')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Distribution of Predicted Maize Yields')
plt.show()

In [ ]:
static_var=['nitrogen','texture_class','lon','lat','yield_mean','tas', 'tasmax', 'tasmin', 'pr', 'rsds']

# For Maize
_, _, maize_test_loader = get_training_dataloader(
    'maize', 'train', mypath,
    region=['USA', 'Other', 'South America', 'Europe'],
    static_var=static_var
)

# For Wheat
_, _, wheat_test_loader = get_training_dataloader(
    'wheat', 'train', mypath,
    region=['USA', 'Other', 'South America', 'Europe'],
    static_var=static_var
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

def evaluate_lstm(model, test_loader, title_prefix=""):
    model.eval()
    y_true_all, y_pred_all = [], []

    with torch.no_grad():
        for x, x_static, y_true in test_loader:
            y_pred = model(x.to(DEVICE), x_static.to(DEVICE)).cpu().numpy()
            y_true = y_true.cpu().numpy()
            y_true_all.append(y_true)
            y_pred_all.append(y_pred)

    y_true_all = np.concatenate(y_true_all)
    y_pred_all = np.concatenate(y_pred_all)
    residuals = y_true_all - y_pred_all
    r2 = r2_score(y_true_all, y_pred_all)
    rmse = np.sqrt(mean_squared_error(y_true_all, y_pred_all))

    return y_true_all, y_pred_all, residuals, r2, rmse

# --- Evaluate both models ---
y_true_wheat, y_pred_wheat, resid_wheat, r2_wheat, rmse_wheat = evaluate_lstm(wheat_unscale_model_GLOB2, wheat_test_loader, "Wheat")
y_true_maize, y_pred_maize, resid_maize, r2_maize, rmse_maize = evaluate_lstm(maize_unscale_model_GLOB, maize_test_loader, "Maize")

# --- Plotting ---
fig, axs = plt.subplots(2, 2, figsize=(14, 8))

# 1. Wheat: Pred vs Actual
axs[0, 0].scatter(y_true_wheat, y_pred_wheat, alpha=0.5, s=15, edgecolor='k')
axs[0, 0].plot([y_true_wheat.min(), y_true_wheat.max()],
               [y_true_wheat.min(), y_true_wheat.max()],
               'r--', label='Perfect Prediction')
axs[0, 0].set_title(f'LSTM Predicted vs Actual Wheat Yield\nR² = {r2_wheat:.4f}, RMSE = {rmse_wheat:.3f}')
axs[0, 0].set_xlabel("Actual Yield")
axs[0, 0].set_ylabel("Predicted Yield")
axs[0, 0].legend()
axs[0, 0].grid(True)

# 2. Maize: Pred vs Actual
axs[0, 1].scatter(y_true_maize, y_pred_maize, alpha=0.5, s=15, edgecolor='k')
axs[0, 1].plot([y_true_maize.min(), y_true_maize.max()],
               [y_true_maize.min(), y_true_maize.max()],
               'r--', label='Perfect Prediction')
axs[0, 1].set_title(f'LSTM Predicted vs Actual Maize Yield\nR² = {r2_maize:.4f}, RMSE = {rmse_maize:.3f}')
axs[0, 1].set_xlabel("Actual Yield")
axs[0, 1].set_ylabel("Predicted Yield")
axs[0, 1].legend()
axs[0, 1].grid(True)

from scipy.stats import gaussian_kde

# 3.
resid_w = resid_wheat.ravel()
mean_w, std_w = np.mean(resid_w), np.std(resid_w)
x_w = np.linspace(resid_w.min(), resid_w.max(), 500)
pdf_w = (1 / (std_w * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_w - mean_w) / std_w) ** 2)

axs[1, 0].hist(resid_w, bins=50, color='forestgreen', edgecolor='black', alpha=0.7, density=True)
axs[1, 0].set_title('LSTM Residuals (Wheat): Actual - Predicted')
axs[1, 0].set_xlabel("Residual")
axs[1, 0].set_ylabel("Density")
axs[1, 0].legend()
axs[1, 0].grid(True)

# 4.
resid_m = resid_maize.ravel()
mean_m, std_m = np.mean(resid_m), np.std(resid_m)
x_m = np.linspace(resid_m.min(), resid_m.max(), 500)
pdf_m = (1 / (std_m * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_m - mean_m) / std_m) ** 2)

axs[1, 1].hist(resid_m, bins=50, color='darkgreen', edgecolor='black', alpha=0.7, density=True)
axs[1, 1].set_title('LSTM Residuals (Maize): Actual - Predicted')
axs[1, 1].set_xlabel("Residual")
axs[1, 1].set_ylabel("Density")
axs[1, 1].legend()
axs[1, 1].grid(True)

plt.tight_layout()
plt.show()

## LSTM with Raw Data

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
BASE = '/content/drive/MyDrive/the-future-crop-challenge'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Running on", device)

In [ ]:
# 1) Mount & imports

import os
import pandas as pd

# 3) List all the maize‐train parquet files
all_files = sorted(fn
                   for fn in os.listdir(BASE)
                   if fn.endswith('_maize_train.parquet'))
print("Found maize‐train files:\n", all_files)

# 4) Load yields (train_solutions) and reset ID from index → column
yld = pd.read_parquet(os.path.join(BASE, 'train_solutions_maize.parquet'))
if yld.index.name == 'ID':
    yld = yld.reset_index()
print("Yields columns:", yld.columns.tolist())

# 5) Load soil+CO₂ and reset ID
soil_fn = [f for f in all_files if f.startswith('soil_co2')][0]
soil   = pd.read_parquet(os.path.join(BASE, soil_fn))
if soil.index.name == 'ID':
    soil = soil.reset_index()
print("Soil columns:", soil.columns.tolist())

# 6) Merge yields + soil on ID
data = yld.merge(
    soil[['ID','real_year','texture_class','nitrogen','co2']],
    on='ID', how='left'
)
print("After soil merge:", data.shape)

# 7) Pick out just the climate files (pr, rsds, tas, tasmax, tasmin)
climate_files = [f for f in all_files
                 if f.split('_')[0] in {'pr','rsds','tas','tasmax','tasmin'}]
print("Will merge these climate files:\n", climate_files)

# 8) Loop & merge, renaming day‑cols to <var>_<day> so there are no dupes
for fn in climate_files:
    var = fn.split('_')[0]                 # 'pr' or 'rsds' or 'tas'/'tasmax'/'tasmin'
    df  = pd.read_parquet(os.path.join(BASE, fn))
    if df.index.name == 'ID':
        df = df.reset_index()

    # day‑columns are named '0','1',…'239'
    day_cols = [c for c in df.columns if c.isdigit()]
    # build rename map so pr‑days become pr_0, pr_1, … etc.
    rename_map = {c: f"{var}_{c}" for c in day_cols}

    # slice out ID + day‑cols, rename, and merge
    df_days = df[['ID'] + day_cols].rename(columns=rename_map)
    print(f"Merging {fn!r}: {len(day_cols)} days → new cols prefix '{var}_'")
    data = data.merge(df_days, on='ID', how='left')

# 9) Check final size
print("Final data shape:", data.shape)
print("Sample of all columns:", data.columns[:12].tolist(), "…")

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

# 1) Train/validation split by real_year
train_df = data[data.real_year < 2020].reset_index(drop=True)
val_df   = data[data.real_year == 2020].reset_index(drop=True)

# 2) Extract target arrays
y_train = train_df['yield'].values.astype(np.float32)
y_val   = val_df  ['yield'].values.astype(np.float32)

# 3) Build dynamic (time‑series) feature arrays
clim_vars = ['pr', 'rsds', 'tas', 'tasmax', 'tasmin']
dyn_cols = []
for var in clim_vars:
    dyn_cols += [c for c in train_df.columns if c.startswith(f"{var}_")]

X_train_dyn = train_df[dyn_cols].values.astype(np.float32)
X_val_dyn   = val_df  [dyn_cols].values.astype(np.float32)

# 4) Scale and reshape dynamic features → (samples, 240, 5)
scaler_dyn   = StandardScaler()
X_train_dyn  = scaler_dyn.fit_transform(X_train_dyn)
X_val_dyn    = scaler_dyn.transform(X_val_dyn)
X_train_dyn  = X_train_dyn.reshape(-1, 240, len(clim_vars))
X_val_dyn    = X_val_dyn.reshape(-1, 240, len(clim_vars))

# 5) Build static feature arrays and scale → (samples, 3)
stat_cols   = ['texture_class', 'nitrogen', 'co2']
X_train_stat = train_df[stat_cols].values.astype(np.float32)
X_val_stat   = val_df  [stat_cols].values.astype(np.float32)

scaler_stat  = StandardScaler()
X_train_stat = scaler_stat.fit_transform(X_train_stat)
X_val_stat   = scaler_stat.transform(X_val_stat)

# 6) Define a PyTorch Dataset
class CropDataset(Dataset):
    def __init__(self, Xdyn, Xstat, y):
        self.Xdyn  = torch.from_numpy(Xdyn)
        self.Xstat = torch.from_numpy(Xstat)
        self.y     = torch.from_numpy(y).unsqueeze(1)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.Xdyn[idx], self.Xstat[idx], self.y[idx]

# 7) Instantiate datasets and dataloaders
train_ds = CropDataset(X_train_dyn, X_train_stat, y_train)
val_ds   = CropDataset(X_val_dyn,   X_val_stat,   y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=2)

# Quick sanity check
print("Train batches:", len(train_loader),
      "| Val batches:", len(val_loader))

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import r2_score
import torch
import torch.nn as nn
from torch.optim import Adam

# 1) Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 2) Model definition
class CropLSTM(nn.Module):
    def __init__(self, n_dyn_feats=5, seq_len=240, n_stat=3):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_dyn_feats, hidden_size=64,
                            batch_first=True)
        self.dyn_fc = nn.Sequential(nn.Linear(64, 32), nn.ReLU())
        self.stat_fc = nn.Sequential(nn.Linear(n_stat, 16), nn.ReLU())
        self.head = nn.Sequential(
            nn.Linear(32 + 16, 16), nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x_dyn, x_stat):
        _, (h_n, _) = self.lstm(x_dyn)      # (1, B, 64)
        h = h_n.squeeze(0)                  # → (B, 64)
        d = self.dyn_fc(h)                  # → (B, 32)
        s = self.stat_fc(x_stat)            # → (B, 16)
        return self.head(torch.cat([d, s], dim=1))

model = CropLSTM(n_dyn_feats=len(clim_vars), n_stat=len(stat_cols)).to(device)

# 3) Loss & optimizer
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)

# 4) Training loop with tqdm & R²
best_val_loss = float('inf')
patience, wait = 5, 0

for epoch in range(1, 51):
    # —— Train ——
    model.train()
    train_losses, train_preds, train_trues = [], [], []
    for Xd, Xs, y in tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False):
        Xd, Xs, y = Xd.to(device), Xs.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(Xd, Xs)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item() * y.size(0))
        train_preds.append(out.detach().cpu())
        train_trues.append(y.cpu())

    train_mse = sum(train_losses) / len(train_loader.dataset)
    train_r2  = r2_score(
        torch.cat(train_trues).numpy(),
        torch.cat(train_preds).numpy()
    )

    # —— Validate ——
    model.eval()
    val_losses, val_preds, val_trues = [], [], []
    for Xd, Xs, y in tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False):
        Xd, Xs, y = Xd.to(device), Xs.to(device), y.to(device)
        out = model(Xd, Xs)
        val_losses.append(criterion(out, y).item() * y.size(0))
        val_preds.append(out.detach().cpu())
        val_trues.append(y.cpu())

    val_mse = sum(val_losses) / len(val_loader.dataset)
    val_r2  = r2_score(
        torch.cat(val_trues).numpy(),
        torch.cat(val_preds).numpy()
    )

    print(f"Epoch {epoch:02d} | "
          f"Train MSE={train_mse:.4f} | Train R²={train_r2:.4f} | "
          f"Val MSE={val_mse:.4f} | Val R²={val_r2:.4f}")

    # Early stopping
    if val_mse < best_val_loss:
        best_val_loss, wait = val_mse, 0
        torch.save(model.state_dict(), 'best_crop_lstm.pth')
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

# 5) Final 2020 evaluation
model.load_state_dict(torch.load('best_crop_lstm.pth'))
model.eval()
all_preds, all_trues = [], []
with torch.no_grad():
    for Xd, Xs, y in val_loader:
        Xd, Xs, y = Xd.to(device), Xs.to(device), y.to(device)
        out = model(Xd, Xs)
        all_preds.append(out.cpu())
        all_trues.append(y.cpu())

all_preds = torch.cat(all_preds).numpy()
all_trues = torch.cat(all_trues).numpy()
final_mse = ((all_preds - all_trues)**2).mean()
final_r2  = r2_score(all_trues, all_preds)

print(f"\nFinal 2020 RMSE: {final_mse**0.5:.4f} | Final 2020 R²: {final_r2:.4f}")

## Baseline MSE of Prediciting Mean of Maize

In [ ]:
import numpy as np

y = train_df['yield'].values
baseline_mse  = np.var(y)                   # MSE of always‑predicting y.mean()
baseline_rmse = np.sqrt(baseline_mse)
print(f"Baseline RMSE = {baseline_rmse:.3f}, MSE = {baseline_mse:.3f}")

## Wheat

In [ ]:
from google.colab import drive
import os, pandas as pd, numpy as np
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader


# 2) Set crop type
crop = 'wheat'

# 3) Discover available wheat‐train files
all_files = sorted(fn for fn in os.listdir(BASE) if fn.endswith(f'_{crop}_train.parquet'))
print("Available wheat‐train files:", all_files)

# 4) Load yields & reset index
yld = pd.read_parquet(os.path.join(BASE, f'train_solutions_{crop}.parquet'))
if yld.index.name == 'ID':
    yld = yld.reset_index()
print("Yields columns:", yld.columns.tolist())

# 5) Load soil/CO₂ & reset index
soil = pd.read_parquet(os.path.join(BASE, f'soil_co2_{crop}_train.parquet'))
if soil.index.name == 'ID':
    soil = soil.reset_index()
print("Soil columns:", soil.columns.tolist())

# 6) Merge yields + soil
data = yld.merge(
    soil[['ID','real_year','texture_class','nitrogen','co2']],
    on='ID', how='left'
)
print("After soil merge:", data.shape)

# 7) Merge each climate series (pr, rsds, tas, tasmax, tasmin)
clim_vars = ['pr','rsds','tas','tasmax','tasmin']
for var in clim_vars:
    fn = f"{var}_{crop}_train.parquet"
    df = pd.read_parquet(os.path.join(BASE, fn))
    if df.index.name == 'ID':
        df = df.reset_index()
    day_cols = [c for c in df.columns if c.isdigit()]
    # rename '0'→'pr_0', etc.
    rename_map = {c: f"{var}_{c}" for c in day_cols}
    df_days = df[['ID'] + day_cols].rename(columns=rename_map)
    data = data.merge(df_days, on='ID', how='left')
    print(f"Merged {fn} → now data shape {data.shape}")

# 8) Train/val split by real_year
train_df = data[data.real_year < 2020].reset_index(drop=True)
val_df   = data[data.real_year == 2020].reset_index(drop=True)

# 9) Targets
y_train = train_df['yield'].values.astype(np.float32)
y_val   = val_df  ['yield'].values.astype(np.float32)

# 10) Dynamic ("time‑series") features
dyn_cols = [c for var in clim_vars for c in train_df.columns if c.startswith(f"{var}_")]
Xtr_dyn = train_df[dyn_cols].values.astype(np.float32)
Xvl_dyn = val_df  [dyn_cols].values.astype(np.float32)

scaler_dyn = StandardScaler()
Xtr_dyn = scaler_dyn.fit_transform(Xtr_dyn).reshape(-1, 240, len(clim_vars))
Xvl_dyn = scaler_dyn.transform(Xvl_dyn).reshape(-1, 240, len(clim_vars))

# 11) Static features
stat_cols = ['texture_class','nitrogen','co2']
Xtr_stat  = train_df[stat_cols].values.astype(np.float32)
Xvl_stat  = val_df  [stat_cols].values.astype(np.float32)

scaler_stat = StandardScaler()
Xtr_stat = scaler_stat.fit_transform(Xtr_stat)
Xvl_stat = scaler_stat.transform(Xvl_stat)

# 12) PyTorch Dataset & DataLoader
class WheatDataset(Dataset):
    def __init__(self, Xdyn, Xstat, y):
        self.Xdyn  = torch.from_numpy(Xdyn)
        self.Xstat = torch.from_numpy(Xstat)
        self.y     = torch.from_numpy(y).unsqueeze(1)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.Xdyn[idx], self.Xstat[idx], self.y[idx]

train_ds = WheatDataset(Xtr_dyn, Xtr_stat, y_train)
val_ds   = WheatDataset(Xvl_dyn, Xvl_stat, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=2)

print("Wheat train batches:", len(train_loader), "| Wheat val batches:", len(val_loader))

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm.auto import tqdm
from sklearn.metrics import r2_score

# ——— 1) Device ———
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ——— 2) Model ———
# Re‑use the CropLSTM class you defined earlier
model = CropLSTM(n_dyn_feats=len(clim_vars), n_stat=len(stat_cols)).to(device)

# ——— 3) Loss & Optimizer ———
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)

# ——— 4) Training loop with early stopping & R² ———
best_val_loss = float('inf')
patience, wait = 5, 0

for epoch in range(1, 51):
    # — Train —
    model.train()
    train_losses, train_preds, train_trues = [], [], []
    for Xd, Xs, y in tqdm(train_loader, desc=f"Epoch {epoch} Train", leave=False):
        Xd, Xs, y = Xd.to(device), Xs.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(Xd, Xs)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item() * y.size(0))
        train_preds.append(out.detach().cpu())
        train_trues.append(y.cpu())

    train_mse = sum(train_losses) / len(train_loader.dataset)
    train_r2  = r2_score(
        torch.cat(train_trues).numpy(),
        torch.cat(train_preds).numpy()
    )

    # — Validate —
    model.eval()
    val_losses, val_preds, val_trues = [], [], []
    for Xd, Xs, y in tqdm(val_loader, desc=f"Epoch {epoch} Val", leave=False):
        Xd, Xs, y = Xd.to(device), Xs.to(device), y.to(device)
        out = model(Xd, Xs)
        val_losses.append(criterion(out, y).item() * y.size(0))
        val_preds.append(out.detach().cpu())
        val_trues.append(y.cpu())

    val_mse = sum(val_losses) / len(val_loader.dataset)
    val_r2  = r2_score(
        torch.cat(val_trues).numpy(),
        torch.cat(val_preds).numpy()
    )

    print(
        f"Epoch {epoch:02d} | "
        f"Train MSE={train_mse:.4f} | Train R²={train_r2:.4f} | "
        f"Val MSE={val_mse:.4f} | Val R²={val_r2:.4f}"
    )

    # — Early stopping —
    if val_mse < best_val_loss:
        best_val_loss, wait = val_mse, 0
        torch.save(model.state_dict(), 'best_wheat_lstm.pth')
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping triggered.")
            break

# ——— 5) Final hold‑out evaluation on 2020 ———
model.load_state_dict(torch.load('best_wheat_lstm.pth'))
model.eval()

all_preds, all_trues = [], []
with torch.no_grad():
    for Xd, Xs, y in val_loader:
        Xd, Xs, y = Xd.to(device), Xs.to(device), y.to(device)
        out = model(Xd, Xs)
        all_preds.append(out.cpu())
        all_trues.append(y.cpu())

all_preds = torch.cat(all_preds).numpy().ravel()
all_trues = torch.cat(all_trues).numpy().ravel()

final_mse = ((all_preds - all_trues)**2).mean()
final_r2  = r2_score(all_trues, all_preds)

print(f"\nFinal 2020 RMSE: {final_mse**0.5:.4f} | Final 2020 R²: {final_r2:.4f}")

In [ ]:
# We check for the baseline RMSE for wheat as well
import numpy as np

# 1) Obtain training yields
y_train = train_df['yield'].values.astype(np.float32)

# 2) Compute baseline MSE & RMSE (predicting the training mean)
baseline_mse  = np.var(y_train)
baseline_rmse = np.sqrt(baseline_mse)

# 3) Print
print(f"Baseline MSE (wheat)  = {baseline_mse:.4f}")
print(f"Baseline RMSE (wheat) = {baseline_rmse:.4f}")

## Plotting

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Predicted vs Actual Scatter Plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=all_trues, y=all_preds, alpha=0.6, edgecolor='k', s=30)
plt.plot([all_trues.min(), all_trues.max()],
         [all_trues.min(), all_trues.max()],
         'r--', label='Perfect Prediction')
plt.xlabel("Actual Yield (2020)")
plt.ylabel("Predicted Yield (2020)")
plt.title(f"Predicted vs Actual Wheat Yield (2020)\nR² = {final_r2:.4f}, RMSE = {final_mse**0.5:.3f}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2. Residuals Histogram
residuals = all_trues - all_preds
plt.figure(figsize=(8, 5))
sns.histplot(residuals, bins=50, kde=True, color='forestgreen')
plt.axvline(0, color='red', linestyle='--')
plt.title("Residuals Distribution (Actual - Predicted): wheat")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error


maize_preds = all_preds.copy()
maize_trues = all_trues.copy()

# Compute R² and RMSE
final_r2_maize = r2_score(maize_trues, maize_preds)
final_rmse_maize = mean_squared_error(maize_trues, maize_preds)

# 1. Predicted vs Actual Scatter Plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=maize_trues, y=maize_preds, alpha=0.6, edgecolor='k', s=30)
plt.plot([maize_trues.min(), maize_trues.max()],
         [maize_trues.min(), maize_trues.max()],
         'r--', label='Perfect Prediction')
plt.xlabel("Actual Yield (2020)")
plt.ylabel("Predicted Yield (2020)")
plt.title(f"Predicted vs Actual Maize Yield (2020)\nR² = {final_r2_maize:.4f}, RMSE = {final_rmse_maize:.3f}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2. Residuals Histogram
residuals_maize = maize_trues - maize_preds
plt.figure(figsize=(8, 5))
sns.histplot(residuals_maize, bins=50, kde=True, color='cornflowerblue')
plt.axvline(0, color='red', linestyle='--')
plt.title("Residuals Distribution (Actual - Predicted): Maize")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.tight_layout()
plt.grid(True)
plt.show()